# setup

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%%capture
!pip install git+https://github.com/EleutherAI/sae.git

In [3]:
# you should load this before cloning repo files
# from .config import SaeConfig
# from .utils import decoder_impl

from sae.config import SaeConfig
from sae.utils import decoder_impl
from sae import Sae

Triton not installed, using eager implementation of SAE decoder.


In [4]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import json
from fnmatch import fnmatch
from pathlib import Path
from typing import NamedTuple, Optional, Callable, Union, List, Tuple
# from jaxtyping import Float, Int

import einops
import torch
from torch import Tensor, nn
from huggingface_hub import snapshot_download
from natsort import natsorted
from safetensors.torch import load_model, save_model

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
from collections import Counter

## corr fns

In [6]:
def batched_correlation(reshaped_activations_A, reshaped_activations_B, batch_size=100):
    # Ensure tensors are on GPU
    if torch.cuda.is_available():
        reshaped_activations_A = reshaped_activations_A.to('cuda')
        reshaped_activations_B = reshaped_activations_B.to('cuda')

    # Normalize columns of A
    mean_A = reshaped_activations_A.mean(dim=0, keepdim=True)
    std_A = reshaped_activations_A.std(dim=0, keepdim=True)
    normalized_A = (reshaped_activations_A - mean_A) / (std_A + 1e-8)  # Avoid division by zero

    # Normalize columns of B
    mean_B = reshaped_activations_B.mean(dim=0, keepdim=True)
    std_B = reshaped_activations_B.std(dim=0, keepdim=True)
    normalized_B = (reshaped_activations_B - mean_B) / (std_B + 1e-8)  # Avoid division by zero

    num_batches = (normalized_B.shape[1] + batch_size - 1) // batch_size
    max_values = []
    max_indices = []

    for batch in range(num_batches):
        start = batch * batch_size
        end = min(start + batch_size, normalized_B.shape[1])
        batch_corr_matrix = torch.matmul(normalized_A.t(), normalized_B[:, start:end]) / normalized_A.shape[0]
        max_val, max_idx = batch_corr_matrix.max(dim=0)
        max_values.append(max_val)
        # max_indices.append(max_idx + start)  # Adjust indices for the batch offset
        max_indices.append(max_idx)  # Adjust indices for the batch offset

        del batch_corr_matrix
        torch.cuda.empty_cache()

    return torch.cat(max_indices), torch.cat(max_values)

## sim fns

In [7]:
import functools
from typing import Any, Callable, Dict, List, Tuple, Union

import numpy as np
import numpy.typing as npt
import torch


def to_numpy_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[npt.NDArray]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> npt.NDArray:
        return x if isinstance(x, np.ndarray) else x.numpy()

    return list(map(convert, args))


def to_torch_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[torch.Tensor]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
        return x if isinstance(x, torch.Tensor) else torch.from_numpy(x)

    return list(map(convert, args))


def adjust_dimensionality(
    R: npt.NDArray, Rp: npt.NDArray, strategy="zero_pad"
) -> Tuple[npt.NDArray, npt.NDArray]:
    D = R.shape[1]
    Dp = Rp.shape[1]
    if strategy == "zero_pad":
        if D - Dp == 0:
            return R, Rp
        elif D - Dp > 0:
            return R, np.concatenate((Rp, np.zeros((Rp.shape[0], D - Dp))), axis=1)
        else:
            return np.concatenate((R, np.zeros((R.shape[0], Dp - D))), axis=1), Rp
    else:
        raise NotImplementedError()


def center_columns(R: npt.NDArray) -> npt.NDArray:
    return R - R.mean(axis=0)[None, :]


def normalize_matrix_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord="fro")


def sim_random_baseline(
    rep1: torch.Tensor, rep2: torch.Tensor, sim_func: Callable, n_permutations: int = 10
) -> Dict[str, Any]:
    torch.manual_seed(1234)
    scores = []
    for _ in range(n_permutations):
        perm = torch.randperm(rep1.size(0))

        score = sim_func(rep1[perm, :], rep2)
        score = score if isinstance(score, float) else score["score"]

        scores.append(score)

    return {"baseline_scores": np.array(scores)}


class Pipeline:
    def __init__(
        self,
        preprocess_funcs: List[Callable[[npt.NDArray], npt.NDArray]],
        similarity_func: Callable[[npt.NDArray, npt.NDArray], Dict[str, Any]],
    ) -> None:
        self.preprocess_funcs = preprocess_funcs
        self.similarity_func = similarity_func

    def __call__(self, R: npt.NDArray, Rp: npt.NDArray) -> Dict[str, Any]:
        for preprocess_func in self.preprocess_funcs:
            R = preprocess_func(R)
            Rp = preprocess_func(Rp)
        return self.similarity_func(R, Rp)

    def __str__(self) -> str:
        def func_name(func: Callable) -> str:
            return (
                func.__name__
                if not isinstance(func, functools.partial)
                else func.func.__name__
            )

        def partial_keywords(func: Callable) -> str:
            if not isinstance(func, functools.partial):
                return ""
            else:
                return str(func.keywords)

        return (
            "Pipeline("
            + (
                "+".join(map(func_name, self.preprocess_funcs))
                + "+"
                + func_name(self.similarity_func)
                + partial_keywords(self.similarity_func)
            )
            + ")"
        )

In [8]:
from typing import List, Set, Union

import numpy as np
import numpy.typing as npt
import sklearn.neighbors
import torch

# from llmcomp.measures.utils import to_numpy_if_needed


def _jac_sim_i(idx_R: Set[int], idx_Rp: Set[int]) -> float:
    return len(idx_R.intersection(idx_Rp)) / len(idx_R.union(idx_Rp))


def jaccard_similarity(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    k: int = 10,
    inner: str = "cosine",
    n_jobs: int = 8,
) -> float:
    R, Rp = to_numpy_if_needed(R, Rp)

    indices_R = nn_array_to_setlist(top_k_neighbors(R, k, inner, n_jobs))
    indices_Rp = nn_array_to_setlist(top_k_neighbors(Rp, k, inner, n_jobs))

    return float(
        np.mean(
            [_jac_sim_i(idx_R, idx_Rp) for idx_R, idx_Rp in zip(indices_R, indices_Rp)]
        )
    )


def top_k_neighbors(
    R: npt.NDArray,
    k: int,
    inner: str,
    n_jobs: int,
) -> npt.NDArray:
    # k+1 nearest neighbors, because we pass in all the data, which means that a point
    # will be the nearest neighbor to itself. We remove this point from the results and
    # report only the k nearest neighbors distinct from the point itself.
    nns = sklearn.neighbors.NearestNeighbors(
        n_neighbors=k + 1, metric=inner, n_jobs=n_jobs
    )
    nns.fit(R)
    _, nns = nns.kneighbors(R)
    return nns[:, 1:]


def nn_array_to_setlist(nn: npt.NDArray) -> List[Set[int]]:
    return [set(idx) for idx in nn]

In [9]:
import functools
import logging
from abc import ABC
from abc import abstractmethod
from dataclasses import dataclass
from dataclasses import field
from typing import Any
from typing import Callable
from typing import get_args
from typing import List
from typing import Literal
from typing import Optional
from typing import Protocol
from typing import Tuple
from typing import Union

import numpy as np
import numpy.typing as npt
import torch
from einops import rearrange
# from loguru import logger

log = logging.getLogger(__name__)


SHAPE_TYPE = Literal["nd", "ntd", "nchw"]

ND_SHAPE, NTD_SHAPE, NCHW_SHAPE = get_args(SHAPE_TYPE)[0], get_args(SHAPE_TYPE)[1], get_args(SHAPE_TYPE)[2]


class SimilarityFunction(Protocol):
    def __call__(  # noqa: E704
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
    ) -> float: ...


class RSMSimilarityFunction(Protocol):
    def __call__(  # noqa: E704
        self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE, n_jobs: int
    ) -> float: ...


@dataclass
class BaseSimilarityMeasure(ABC):
    larger_is_more_similar: bool
    is_symmetric: bool

    is_metric: bool | None = None
    invariant_to_affine: bool | None = None
    invariant_to_invertible_linear: bool | None = None
    invariant_to_ortho: bool | None = None
    invariant_to_permutation: bool | None = None
    invariant_to_isotropic_scaling: bool | None = None
    invariant_to_translation: bool | None = None
    name: str = field(init=False)

    def __post_init__(self):
        self.name = self.__class__.__name__

    @abstractmethod
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        raise NotImplementedError


class FunctionalSimilarityMeasure(BaseSimilarityMeasure):
    @abstractmethod
    def __call__(self, output_a: torch.Tensor | npt.NDArray, output_b: torch.Tensor | npt.NDArray) -> float:
        raise NotImplementedError


@dataclass(kw_only=True)
class RepresentationalSimilarityMeasure(BaseSimilarityMeasure):
    sim_func: SimilarityFunction

    def __call__(
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
    ) -> float:
        return self.sim_func(R, Rp, shape)


class RSMSimilarityMeasure(RepresentationalSimilarityMeasure):
    sim_func: RSMSimilarityFunction

    @staticmethod
    def estimate_good_number_of_jobs(R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray) -> int:
        # RSMs in are NxN (or DxD) so the number of jobs should roughly scale quadratically with increase in N (or D).
        # False! As long as sklearn-native metrics are used, they will use parallel implementations regardless of job
        # count. Each job would spawn their own threads, which leads to oversubscription of cores and thus slowdown.
        # This seems to be not fully correct (n_jobs=2 seems to actually use two cores), but using n_jobs=1 seems the
        # fastest.
        return 1

    def __call__(
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
        n_jobs: Optional[int] = None,
    ) -> float:
        if n_jobs is None:
            n_jobs = self.estimate_good_number_of_jobs(R, Rp)
        return self.sim_func(R, Rp, shape, n_jobs=n_jobs)


def to_numpy_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[npt.NDArray]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> npt.NDArray:
        return x if isinstance(x, np.ndarray) else x.numpy()

    return list(map(convert, args))


def to_torch_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[torch.Tensor]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
        return x if isinstance(x, torch.Tensor) else torch.from_numpy(x)

    return list(map(convert, args))


def adjust_dimensionality(R: npt.NDArray, Rp: npt.NDArray, strategy="zero_pad") -> Tuple[npt.NDArray, npt.NDArray]:
    D = R.shape[1]
    Dp = Rp.shape[1]
    if strategy == "zero_pad":
        if D - Dp == 0:
            return R, Rp
        elif D - Dp > 0:
            return R, np.concatenate((Rp, np.zeros((Rp.shape[0], D - Dp))), axis=1)
        else:
            return np.concatenate((R, np.zeros((R.shape[0], Dp - D))), axis=1), Rp
    else:
        raise NotImplementedError()


def center_columns(R: npt.NDArray) -> npt.NDArray:
    return R - R.mean(axis=0)[None, :]


def normalize_matrix_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord="fro")


def normalize_row_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord=2, axis=1, keepdims=True)


def standardize(R: npt.NDArray) -> npt.NDArray:
    return (R - R.mean(axis=0, keepdims=True)) / R.std(axis=0)


def double_center(x: npt.NDArray) -> npt.NDArray:
    return x - x.mean(axis=0, keepdims=True) - x.mean(axis=1, keepdims=True) + x.mean()


def align_spatial_dimensions(R: npt.NDArray, Rp: npt.NDArray) -> Tuple[npt.NDArray, npt.NDArray]:
    """
    Aligns spatial representations by resizing them to the smallest spatial dimension.
    Subsequent aligned spatial representations are flattened, with the spatial aligned representations
    moving into the *sample* dimension.
    """
    R_re, Rp_re = resize_wh_reps(R, Rp)
    R_re = rearrange(R_re, "n c h w -> (n h w) c")
    Rp_re = rearrange(Rp_re, "n c h w -> (n h w) c")
    if R_re.shape[0] > 5000:
        logger.info(f"Got {R_re.shape[0]} samples in N after flattening. Subsampling to reduce compute.")
        subsample = R_re.shape[0] // 5000
        R_re = R_re[::subsample]
        Rp_re = Rp_re[::subsample]

    return R_re, Rp_re


def average_pool_downsample(R, resize: bool, new_size: tuple[int, int]):
    if not resize:
        return R  # do nothing
    else:
        is_numpy = isinstance(R, np.ndarray)
        R_torch = torch.from_numpy(R) if is_numpy else R
        R_torch = torch.nn.functional.adaptive_avg_pool2d(R_torch, new_size)
        return R_torch.numpy() if is_numpy else R_torch


def resize_wh_reps(R: npt.NDArray, Rp: npt.NDArray) -> Tuple[npt.NDArray, npt.NDArray]:
    """
    Function for resizing spatial representations that are not the same size.
    Does through fourier transform and resizing.

    Args:
        R: numpy array of shape  [batch_size, height, width, num_channels]
        RP: numpy array of shape [batch_size, height, width, num_channels]

    Returns:
        fft_acts1: numpy array of shape [batch_size, (new) height, (new) width, num_channels]
        fft_acts2: numpy array of shape [batch_size, (new) height, (new) width, num_channels]

    """
    height1, width1 = R.shape[2], R.shape[3]
    height2, width2 = Rp.shape[2], Rp.shape[3]
    if height1 != height2 or width1 != width2:
        height = min(height1, height2)
        width = min(width1, width2)
        new_size = [height, width]
        resize = True
    else:
        height = height1
        width = width1
        new_size = None
        resize = False

    # resize and preprocess with fft
    avg_ds1 = average_pool_downsample(R, resize=resize, new_size=new_size)
    avg_ds2 = average_pool_downsample(Rp, resize=resize, new_size=new_size)
    return avg_ds1, avg_ds2


def fft_resize(images, resize=False, new_size=None):
    """Function for applying DFT and resizing.

    This function takes in an array of images, applies the 2-d fourier transform
    and resizes them according to new_size, keeping the frequencies that overlap
    between the two sizes.

    Args:
              images: a numpy array with shape
                      [batch_size, height, width, num_channels]
              resize: boolean, whether or not to resize
              new_size: a tuple (size, size), with height and width the same

    Returns:
              im_fft_downsampled: a numpy array with shape
                           [batch_size, (new) height, (new) width, num_channels]
    """
    assert len(images.shape) == 4, "expecting images to be" "[batch_size, height, width, num_channels]"
    if resize:
        # FFT --> remove high frequencies --> inverse FFT
        im_complex = images.astype("complex64")
        im_fft = np.fft.fft2(im_complex, axes=(1, 2))
        im_shifted = np.fft.fftshift(im_fft, axes=(1, 2))

        center_width = im_shifted.shape[2] // 2
        center_height = im_shifted.shape[1] // 2
        half_w = new_size[0] // 2
        half_h = new_size[1] // 2
        cropped_fft = im_shifted[
            :, center_height - half_h : center_height + half_h, center_width - half_w : center_width + half_w, :
        ]
        cropped_fft_shifted_back = np.fft.ifft2(cropped_fft, axes=(1, 2))
        return cropped_fft_shifted_back.real
    else:
        return images


class Pipeline:
    def __init__(
        self,
        preprocess_funcs: List[Callable[[npt.NDArray], npt.NDArray]],
        similarity_func: Callable[[npt.NDArray, npt.NDArray, SHAPE_TYPE], float],
    ) -> None:
        self.preprocess_funcs = preprocess_funcs
        self.similarity_func = similarity_func

    def __call__(self, R: npt.NDArray, Rp: npt.NDArray, shape: SHAPE_TYPE) -> float:
        try:
            for preprocess_func in self.preprocess_funcs:
                R = preprocess_func(R)
                Rp = preprocess_func(Rp)
            return self.similarity_func(R, Rp, shape)
        except ValueError as e:
            log.info(f"Pipeline failed: {e}")
            return np.nan

    def __str__(self) -> str:
        def func_name(func: Callable) -> str:
            return func.__name__ if not isinstance(func, functools.partial) else func.func.__name__

        def partial_keywords(func: Callable) -> str:
            if not isinstance(func, functools.partial):
                return ""
            else:
                return str(func.keywords)

        return (
            "Pipeline("
            + (
                "+".join(map(func_name, self.preprocess_funcs))
                + "+"
                + func_name(self.similarity_func)
                + partial_keywords(self.similarity_func)
            )
            + ")"
        )


def flatten(*args: Union[torch.Tensor, npt.NDArray], shape: SHAPE_TYPE) -> List[Union[torch.Tensor, npt.NDArray]]:
    if shape == "ntd":
        return list(map(flatten_nxtxd_to_ntxd, args))
    elif shape == "nd":
        return list(args)
    elif shape == "nchw":
        return list(map(flatten_nxcxhxw_to_nxchw, args))  # Flattening non-trivial for nchw
    else:
        raise ValueError("Unknown shape of representations. Must be one of 'ntd', 'nchw', 'nd'.")


def flatten_nxtxd_to_ntxd(R: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
    R = to_torch_if_needed(R)[0]
    log.debug("Shape before flattening: %s", str(R.shape))
    R = torch.flatten(R, start_dim=0, end_dim=1)
    log.debug("Shape after flattening: %s", str(R.shape))
    return R


def flatten_nxcxhxw_to_nxchw(R: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
    R = to_torch_if_needed(R)[0]
    log.debug("Shape before flattening: %s", str(R.shape))
    R = torch.reshape(R, (R.shape[0], -1))
    log.debug("Shape after flattening: %s", str(R.shape))
    return R

In [10]:
import scipy.optimize

def permutation_procrustes(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
    optimal_permutation_alignment: Optional[Tuple[npt.NDArray, npt.NDArray]] = None,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    R, Rp = adjust_dimensionality(R, Rp)

    if not optimal_permutation_alignment:
        PR, PRp = scipy.optimize.linear_sum_assignment(R.T @ Rp, maximize=True)  # returns column assignments
        optimal_permutation_alignment = (PR, PRp)
    PR, PRp = optimal_permutation_alignment
    return float(np.linalg.norm(R[:, PR] - Rp[:, PRp], ord="fro"))

In [11]:
from typing import Optional
from typing import Union

import numpy as np
import numpy.typing as npt
import scipy.spatial.distance
import scipy.stats
import sklearn.metrics
import torch
# from repsim.measures.utils import flatten
# from repsim.measures.utils import RSMSimilarityMeasure
# from repsim.measures.utils import SHAPE_TYPE
# from repsim.measures.utils import to_numpy_if_needed


def representational_similarity_analysis(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
    inner="correlation",
    outer="spearman",
    n_jobs: Optional[int] = None,
) -> float:
    """Representational similarity analysis

    Args:
        R (Union[torch.Tensor, npt.NDArray]): N x D representation
        Rp (Union[torch.Tensor, npt.NDArray]): N x D' representation
        inner (str, optional): inner similarity function for RSM. Must be one of
            scipy.spatial.distance.pdist identifiers . Defaults to "correlation".
        outer (str, optional): outer similarity function that compares RSMs. Defaults to
             "spearman". Must be one of "spearman", "euclidean"

    Returns:
        float: _description_
    """
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)

    if inner == "correlation":
        # n_jobs only works if metric is in PAIRWISE_DISTANCES as defined in sklearn, i.e., not for correlation.
        # But correlation = 1 - cosine dist of row-centered data, so we use the faster cosine metric and center the data.
        R = R - R.mean(axis=1, keepdims=True)
        S = scipy.spatial.distance.squareform(  # take the lower triangle of RSM
            1 - sklearn.metrics.pairwise_distances(R, metric="cosine", n_jobs=n_jobs),  # type:ignore
            checks=False,
        )
        Rp = Rp - Rp.mean(axis=1, keepdims=True)
        Sp = scipy.spatial.distance.squareform(
            1 - sklearn.metrics.pairwise_distances(Rp, metric="cosine", n_jobs=n_jobs),  # type:ignore
            checks=False,
        )
    elif inner == "euclidean":
        # take the lower triangle of RSM
        S = scipy.spatial.distance.squareform(
            sklearn.metrics.pairwise_distances(R, metric=inner, n_jobs=n_jobs), checks=False
        )
        Sp = scipy.spatial.distance.squareform(
            sklearn.metrics.pairwise_distances(Rp, metric=inner, n_jobs=n_jobs), checks=False
        )
    else:
        raise NotImplementedError(f"{inner=}")

    if outer == "spearman":
        return scipy.stats.spearmanr(S, Sp).statistic  # type:ignore
    elif outer == "euclidean":
        return float(np.linalg.norm(S - Sp, ord=2))
    else:
        raise ValueError(f"Unknown outer similarity function: {outer}")


class RSA(RSMSimilarityMeasure):
    def __init__(self):
        # choice of inner/outer in __call__ if fixed to default values, so these values are always the same
        super().__init__(
            sim_func=representational_similarity_analysis,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=True,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=False,
            invariant_to_permutation=True,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

In [12]:
##################################################################################
# Copied from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/cca_core.py
# Copyright 2018 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
The core code for applying Canonical Correlation Analysis to deep networks.

This module contains the core functions to apply canonical correlation analysis
to deep neural networks. The main function is get_cca_similarity, which takes in
two sets of activations, typically the neurons in two layers and their outputs
on all of the datapoints D = [d_1,...,d_m] that have been passed through.

Inputs have shape (num_neurons1, m), (num_neurons2, m). This can be directly
applied used on fully connected networks. For convolutional layers, the 3d block
of neurons can either be flattened entirely, along channels, or alternatively,
the dft_ccas (Discrete Fourier Transform) module can be used.

See:
https://arxiv.org/abs/1706.05806
https://arxiv.org/abs/1806.05759
for full details.

"""
import numpy as np
# from repsim.measures.utils import align_spatial_dimensions

num_cca_trials = 5


def positivedef_matrix_sqrt(array):
    """Stable method for computing matrix square roots, supports complex matrices.

    Args:
              array: A numpy 2d array, can be complex valued that is a positive
                     definite symmetric (or hermitian) matrix

    Returns:
              sqrtarray: The matrix square root of array
    """
    w, v = np.linalg.eigh(array)
    #  A - np.dot(v, np.dot(np.diag(w), v.T))
    wsqrt = np.sqrt(w)
    sqrtarray = np.dot(v, np.dot(np.diag(wsqrt), np.conj(v).T))
    return sqrtarray


def remove_small(sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon):
    """Takes covariance between X, Y, and removes values of small magnitude.

    Args:
              sigma_xx: 2d numpy array, variance matrix for x
              sigma_xy: 2d numpy array, crossvariance matrix for x,y
              sigma_yx: 2d numpy array, crossvariance matrixy for x,y,
                        (conjugate) transpose of sigma_xy
              sigma_yy: 2d numpy array, variance matrix for y
              epsilon : cutoff value for norm below which directions are thrown
                         away

    Returns:
              sigma_xx_crop: 2d array with low x norm directions removed
              sigma_xy_crop: 2d array with low x and y norm directions removed
              sigma_yx_crop: 2d array with low x and y norm directiosn removed
              sigma_yy_crop: 2d array with low y norm directions removed
              x_idxs: indexes of sigma_xx that were removed
              y_idxs: indexes of sigma_yy that were removed
    """

    x_diag = np.abs(np.diagonal(sigma_xx))
    y_diag = np.abs(np.diagonal(sigma_yy))
    x_idxs = x_diag >= epsilon
    y_idxs = y_diag >= epsilon

    sigma_xx_crop = sigma_xx[x_idxs][:, x_idxs]
    sigma_xy_crop = sigma_xy[x_idxs][:, y_idxs]
    sigma_yx_crop = sigma_yx[y_idxs][:, x_idxs]
    sigma_yy_crop = sigma_yy[y_idxs][:, y_idxs]

    return (sigma_xx_crop, sigma_xy_crop, sigma_yx_crop, sigma_yy_crop, x_idxs, y_idxs)


def compute_ccas(sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon, verbose=True):
    """Main cca computation function, takes in variances and crossvariances.

    This function takes in the covariances and cross covariances of X, Y,
    preprocesses them (removing small magnitudes) and outputs the raw results of
    the cca computation, including cca directions in a rotated space, and the
    cca correlation coefficient values.

    Args:
              sigma_xx: 2d numpy array, (num_neurons_x, num_neurons_x)
                        variance matrix for x
              sigma_xy: 2d numpy array, (num_neurons_x, num_neurons_y)
                        crossvariance matrix for x,y
              sigma_yx: 2d numpy array, (num_neurons_y, num_neurons_x)
                        crossvariance matrix for x,y (conj) transpose of sigma_xy
              sigma_yy: 2d numpy array, (num_neurons_y, num_neurons_y)
                        variance matrix for y
              epsilon:  small float to help with stabilizing computations
              verbose:  boolean on whether to print intermediate outputs

    Returns:
              [ux, sx, vx]: [numpy 2d array, numpy 1d array, numpy 2d array]
                            ux and vx are (conj) transposes of each other, being
                            the canonical directions in the X subspace.
                            sx is the set of canonical correlation coefficients-
                            how well corresponding directions in vx, Vy correlate
                            with each other.
              [uy, sy, vy]: Same as above, but for Y space
              invsqrt_xx:   Inverse square root of sigma_xx to transform canonical
                            directions back to original space
              invsqrt_yy:   Same as above but for sigma_yy
              x_idxs:       The indexes of the input sigma_xx that were pruned
                            by remove_small
              y_idxs:       Same as above but for sigma_yy
    """

    (sigma_xx, sigma_xy, sigma_yx, sigma_yy, x_idxs, y_idxs) = remove_small(
        sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon
    )

    numx = sigma_xx.shape[0]
    numy = sigma_yy.shape[0]

    if numx == 0 or numy == 0:
        return (
            [0, 0, 0],
            [0, 0, 0],
            np.zeros_like(sigma_xx),
            np.zeros_like(sigma_yy),
            x_idxs,
            y_idxs,
        )

    if verbose:
        print("adding eps to diagonal and taking inverse")
    sigma_xx += epsilon * np.eye(numx)
    sigma_yy += epsilon * np.eye(numy)
    inv_xx = np.linalg.pinv(sigma_xx)
    inv_yy = np.linalg.pinv(sigma_yy)

    if verbose:
        print("taking square root")
    invsqrt_xx = positivedef_matrix_sqrt(inv_xx)
    invsqrt_yy = positivedef_matrix_sqrt(inv_yy)

    if verbose:
        print("dot products...")
    arr = np.dot(invsqrt_xx, np.dot(sigma_xy, invsqrt_yy))

    if verbose:
        print("trying to take final svd")
    u, s, v = np.linalg.svd(arr)

    if verbose:
        print("computed everything!")

    return [u, np.abs(s), v], invsqrt_xx, invsqrt_yy, x_idxs, y_idxs


def sum_threshold(array, threshold):
    """Computes threshold index of decreasing nonnegative array by summing.

    This function takes in a decreasing array nonnegative floats, and a
    threshold between 0 and 1. It returns the index i at which the sum of the
    array up to i is threshold*total mass of the array.

    Args:
              array: a 1d numpy array of decreasing, nonnegative floats
              threshold: a number between 0 and 1

    Returns:
              i: index at which np.sum(array[:i]) >= threshold
    """
    assert (threshold >= 0) and (threshold <= 1), "print incorrect threshold"

    for i in range(len(array)):
        if np.sum(array[:i]) / np.sum(array) >= threshold:
            return i


def create_zero_dict(compute_dirns, dimension):
    """Outputs a zero dict when neuron activation norms too small.

    This function creates a return_dict with appropriately shaped zero entries
    when all neuron activations are very small.

    Args:
              compute_dirns: boolean, whether to have zero vectors for directions
              dimension: int, defines shape of directions

    Returns:
              return_dict: a dict of appropriately shaped zero entries
    """
    return_dict = {}
    return_dict["mean"] = (np.asarray(0), np.asarray(0))
    return_dict["sum"] = (np.asarray(0), np.asarray(0))
    return_dict["cca_coef1"] = np.asarray(0)
    return_dict["cca_coef2"] = np.asarray(0)
    return_dict["idx1"] = 0
    return_dict["idx2"] = 0

    if compute_dirns:
        return_dict["cca_dirns1"] = np.zeros((1, dimension))
        return_dict["cca_dirns2"] = np.zeros((1, dimension))

    return return_dict


def get_cca_similarity(
    acts1,
    acts2,
    epsilon=0.0,
    threshold=0.98,
    compute_coefs=True,
    compute_dirns=False,
    verbose=True,
):
    """The main function for computing cca similarities.

    This function computes the cca similarity between two sets of activations,
    returning a dict with the cca coefficients, a few statistics of the cca
    coefficients, and (optionally) the actual directions.

    Args:
              acts1: (num_neurons1, data_points) a 2d numpy array of neurons by
                     datapoints where entry (i,j) is the output of neuron i on
                     datapoint j.
              acts2: (num_neurons2, data_points) same as above, but (potentially)
                     for a different set of neurons. Note that acts1 and acts2
                     can have different numbers of neurons, but must agree on the
                     number of datapoints

              epsilon: small float to help stabilize computations

              threshold: float between 0, 1 used to get rid of trailing zeros in
                         the cca correlation coefficients to output more accurate
                         summary statistics of correlations.


              compute_coefs: boolean value determining whether coefficients
                             over neurons are computed. Needed for computing
                             directions

              compute_dirns: boolean value determining whether actual cca
                             directions are computed. (For very large neurons and
                             datasets, may be better to compute these on the fly
                             instead of store in memory.)

              verbose: Boolean, whether intermediate outputs are printed

    Returns:
              return_dict: A dictionary with outputs from the cca computations.
                           Contains neuron coefficients (combinations of neurons
                           that correspond to cca directions), the cca correlation
                           coefficients (how well aligned directions correlate),
                           x and y idxs (for computing cca directions on the fly
                           if compute_dirns=False), and summary statistics. If
                           compute_dirns=True, the cca directions are also
                           computed.
    """

    # assert dimensionality equal
    assert acts1.shape[1] == acts2.shape[1], "dimensions don't match"
    # check that acts1, acts2 are transposition
    assert acts1.shape[0] < acts1.shape[1], "input must be number of neurons" "by datapoints"
    return_dict = {}

    # compute covariance with numpy function for extra stability
    numx = acts1.shape[0]
    numy = acts2.shape[0]

    covariance = np.cov(acts1, acts2)
    sigmaxx = covariance[:numx, :numx]
    sigmaxy = covariance[:numx, numx:]
    sigmayx = covariance[numx:, :numx]
    sigmayy = covariance[numx:, numx:]

    # rescale covariance to make cca computation more stable
    xmax = np.max(np.abs(sigmaxx))
    ymax = np.max(np.abs(sigmayy))
    sigmaxx /= xmax
    sigmayy /= ymax
    sigmaxy /= np.sqrt(xmax * ymax)
    sigmayx /= np.sqrt(xmax * ymax)

    ([u, s, v], invsqrt_xx, invsqrt_yy, x_idxs, y_idxs) = compute_ccas(
        sigmaxx, sigmaxy, sigmayx, sigmayy, epsilon=epsilon, verbose=verbose
    )

    # if x_idxs or y_idxs is all false, return_dict has zero entries
    if (not np.any(x_idxs)) or (not np.any(y_idxs)):
        return create_zero_dict(compute_dirns, acts1.shape[1])

    if compute_coefs:
        # also compute full coefficients over all neurons
        x_mask = np.dot(x_idxs.reshape((-1, 1)), x_idxs.reshape((1, -1)))
        y_mask = np.dot(y_idxs.reshape((-1, 1)), y_idxs.reshape((1, -1)))

        return_dict["coef_x"] = u.T
        return_dict["invsqrt_xx"] = invsqrt_xx
        return_dict["full_coef_x"] = np.zeros((numx, numx))
        np.place(return_dict["full_coef_x"], x_mask, return_dict["coef_x"])
        return_dict["full_invsqrt_xx"] = np.zeros((numx, numx))
        np.place(return_dict["full_invsqrt_xx"], x_mask, return_dict["invsqrt_xx"])

        return_dict["coef_y"] = v
        return_dict["invsqrt_yy"] = invsqrt_yy
        return_dict["full_coef_y"] = np.zeros((numy, numy))
        np.place(return_dict["full_coef_y"], y_mask, return_dict["coef_y"])
        return_dict["full_invsqrt_yy"] = np.zeros((numy, numy))
        np.place(return_dict["full_invsqrt_yy"], y_mask, return_dict["invsqrt_yy"])

        # compute means
        neuron_means1 = np.mean(acts1, axis=1, keepdims=True)
        neuron_means2 = np.mean(acts2, axis=1, keepdims=True)
        return_dict["neuron_means1"] = neuron_means1
        return_dict["neuron_means2"] = neuron_means2

    if compute_dirns:
        # orthonormal directions that are CCA directions
        cca_dirns1 = (
            np.dot(
                np.dot(return_dict["full_coef_x"], return_dict["full_invsqrt_xx"]),
                (acts1 - neuron_means1),
            )
            + neuron_means1
        )
        cca_dirns2 = (
            np.dot(
                np.dot(return_dict["full_coef_y"], return_dict["full_invsqrt_yy"]),
                (acts2 - neuron_means2),
            )
            + neuron_means2
        )

    # get rid of trailing zeros in the cca coefficients
    idx1 = sum_threshold(s, threshold)
    idx2 = sum_threshold(s, threshold)

    return_dict["cca_coef1"] = s
    return_dict["cca_coef2"] = s
    return_dict["x_idxs"] = x_idxs
    return_dict["y_idxs"] = y_idxs
    # summary statistics
    return_dict["mean"] = (np.mean(s[:idx1]), np.mean(s[:idx2]))
    return_dict["sum"] = (np.sum(s), np.sum(s))

    if compute_dirns:
        return_dict["cca_dirns1"] = cca_dirns1
        return_dict["cca_dirns2"] = cca_dirns2

    return return_dict


def robust_cca_similarity(acts1, acts2, threshold=0.98, epsilon=1e-6, compute_dirns=True):
    """Calls get_cca_similarity multiple times while adding noise.

    This function is very similar to get_cca_similarity, and can be used if
    get_cca_similarity doesn't converge for some pair of inputs. This function
    adds some noise to the activations to help convergence.

    Args:
              acts1: (num_neurons1, data_points) a 2d numpy array of neurons by
                     datapoints where entry (i,j) is the output of neuron i on
                     datapoint j.
              acts2: (num_neurons2, data_points) same as above, but (potentially)
                     for a different set of neurons. Note that acts1 and acts2
                     can have different numbers of neurons, but must agree on the
                     number of datapoints

              threshold: float between 0, 1 used to get rid of trailing zeros in
                         the cca correlation coefficients to output more accurate
                         summary statistics of correlations.

              epsilon: small float to help stabilize computations

              compute_dirns: boolean value determining whether actual cca
                             directions are computed. (For very large neurons and
                             datasets, may be better to compute these on the fly
                             instead of store in memory.)

    Returns:
              return_dict: A dictionary with outputs from the cca computations.
                           Contains neuron coefficients (combinations of neurons
                           that correspond to cca directions), the cca correlation
                           coefficients (how well aligned directions correlate),
                           x and y idxs (for computing cca directions on the fly
                           if compute_dirns=False), and summary statistics. If
                           compute_dirns=True, the cca directions are also
                           computed.
    """

    for trial in range(num_cca_trials):
        try:
            return_dict = get_cca_similarity(acts1, acts2, threshold, compute_dirns)
        except np.linalg.LinAlgError:
            acts1 = acts1 * 1e-1 + np.random.normal(size=acts1.shape) * epsilon
            acts2 = acts2 * 1e-1 + np.random.normal(size=acts1.shape) * epsilon
            if trial + 1 == num_cca_trials:
                raise

    return return_dict
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/cca_core.py


def top_k_pca_comps(singular_values, threshold=0.99):
    total_variance = np.sum(singular_values**2)
    explained_variance = (singular_values**2) / total_variance
    cumulative_variance = np.cumsum(explained_variance)
    return np.argmax(cumulative_variance >= threshold * total_variance) + 1


def _svcca_original(acts1, acts2):
    # Copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/tutorials/001_Introduction.ipynb
    # Modification: get_cca_similarity is in the same file.
    # Modification: top-k PCA component selection s.t. explained variance > 0.99 total variance
    # Mean subtract activations
    cacts1 = acts1 - np.mean(acts1, axis=1, keepdims=True)
    cacts2 = acts2 - np.mean(acts2, axis=1, keepdims=True)

    # Perform SVD
    U1, s1, V1 = np.linalg.svd(cacts1, full_matrices=False)
    U2, s2, V2 = np.linalg.svd(cacts2, full_matrices=False)

    # top-k PCA components only
    k1 = top_k_pca_comps(s1)
    k2 = top_k_pca_comps(s2)

    svacts1 = np.dot(s1[:k1] * np.eye(k1), V1[:k1])
    # can also compute as svacts1 = np.dot(U1.T[:20], cacts1)
    svacts2 = np.dot(s2[:k2] * np.eye(k2), V2[:k2])
    # can also compute as svacts1 = np.dot(U2.T[:20], cacts2)

    svcca_results = get_cca_similarity(svacts1, svacts2, epsilon=1e-10, verbose=False)
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/tutorials/001_Introduction.ipynb
    return np.mean(svcca_results["cca_coef1"])


# Copied from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/pwcca.py
# Modification: get_cca_similarity is in the same file.
def compute_pwcca(acts1, acts2, epsilon=0.0):
    """Computes projection weighting for weighting CCA coefficients

    Args:
         acts1: 2d numpy array, shaped (neurons, num_datapoints)
         acts2: 2d numpy array, shaped (neurons, num_datapoints)

    Returns:
         Original cca coefficient mean and weighted mean

    """
    sresults = get_cca_similarity(
        acts1,
        acts2,
        epsilon=epsilon,
        compute_dirns=False,
        compute_coefs=True,
        verbose=False,
    )
    if np.sum(sresults["x_idxs"]) <= np.sum(sresults["y_idxs"]):
        dirns = (
            np.dot(
                sresults["coef_x"],
                (acts1[sresults["x_idxs"]] - sresults["neuron_means1"][sresults["x_idxs"]]),
            )
            + sresults["neuron_means1"][sresults["x_idxs"]]
        )
        coefs = sresults["cca_coef1"]
        acts = acts1
        idxs = sresults["x_idxs"]
    else:
        dirns = (
            np.dot(
                sresults["coef_y"],
                (acts1[sresults["y_idxs"]] - sresults["neuron_means2"][sresults["y_idxs"]]),
            )
            + sresults["neuron_means2"][sresults["y_idxs"]]
        )
        coefs = sresults["cca_coef2"]
        acts = acts2
        idxs = sresults["y_idxs"]
    P, _ = np.linalg.qr(dirns.T)
    weights = np.sum(np.abs(np.dot(P.T, acts[idxs].T)), axis=1)
    weights = weights / np.sum(weights)

    return np.sum(weights * coefs), weights, coefs
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/pwcca.py


##################################################################################

from typing import Union  # noqa:e402

import numpy.typing as npt  # noqa:e402
import torch  # noqa:e402

# from repsim.measures.utils import (
#     SHAPE_TYPE,
#     flatten,
#     resize_wh_reps,
#     to_numpy_if_needed,
#     RepresentationalSimilarityMeasure,
# )  # noqa:e402


def svcca(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    return _svcca_original(R.T, Rp.T)


def pwcca(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    return compute_pwcca(R.T, Rp.T)[0]


class SVCCA(RepresentationalSimilarityMeasure):
    def __init__(self):
        super().__init__(
            sim_func=svcca,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=True,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=True,
            invariant_to_permutation=True,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

    def __call__(self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE) -> float:
        if shape == "nchw":
            # Move spatial dimensions into the sample dimension
            # If not the same spatial dimension, resample via FFT.
            R, Rp = align_spatial_dimensions(R, Rp)
            shape = "nd"

        return self.sim_func(R, Rp, shape)


class PWCCA(RepresentationalSimilarityMeasure):
    def __init__(self):
        super().__init__(
            sim_func=pwcca,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=False,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=False,
            invariant_to_permutation=False,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

    def __call__(self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE) -> float:
        if shape == "nchw":
            # Move spatial dimensions into the sample dimension
            # If not the same spatial dimension, resample via FFT.
            R, Rp = align_spatial_dimensions(R, Rp)
            shape = "nd"

        return self.sim_func(R, Rp, shape)

## get rand

In [13]:
def score_rand(num_runs, weight_matrix_np, weight_matrix_2, num_feats, sim_fn, shapereq_bool):
    all_rand_scores = []
    for i in range(num_runs):
        rand_modA_feats = np.random.randint(low=0, high=weight_matrix_np.shape[0], size=num_feats).tolist()
        rand_modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()

        if shapereq_bool:
            score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats], "nd")
        else:
            score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats])
        all_rand_scores.append(score)
    # print(sum(all_rand_scores) / len(all_rand_scores))
    # plt.hist(all_rand_scores)
    # plt.show()
    return sum(all_rand_scores) / len(all_rand_scores)

In [77]:
import random
def shuffle_rand(num_runs, weight_matrix_np, weight_matrix_2, num_feats, sim_fn, shapereq_bool):
    all_rand_scores = []
    for i in range(num_runs):
        row_idxs = list(range(num_feats))
        random.shuffle(row_idxs)
        if shapereq_bool:
            score = sim_fn(weight_matrix_np, weight_matrix_2[row_idxs], "nd")
        else:
            score = sim_fn(weight_matrix_np, weight_matrix_2[row_idxs])
        all_rand_scores.append(score)
    # return sum(all_rand_scores) / len(all_rand_scores)
    return all_rand_scores

In [67]:
def score_rand_corr(num_runs, weight_matrix_np, weight_matrix_2, num_feats, highest_correlations_indices_AB, sim_fn, shapereq_bool):
    all_rand_scores = []
    i = 0
    # for i in range(num_runs):
    while i < num_runs:
        try:
            rand_modB_feats = np.random.choice(range(weight_matrix_2.shape[0]), size=num_feats, replace=False).tolist()
            rand_modA_feats = [highest_correlations_indices_AB[index] for index in rand_modB_feats]

            if shapereq_bool:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats], "nd")
            else:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats])
            all_rand_scores.append(score)
            i += 1
        except:
            continue
    # print(sum(all_rand_scores) / len(all_rand_scores))
    # plt.hist(all_rand_scores)
    # plt.show()
    return all_rand_scores

## plot fns

In [15]:
def plot_svcca_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['svcca_paired'] for i in range(0, 12)]
    unpaired_values = [layer_to_dictscores[i]['svcca_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('SVCCA')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [16]:
def plot_rsa_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['rsa_paired'] for i in range(0, 12)]
    unpaired_values = [layer_to_dictscores[i]['rsa_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('RSA')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [17]:
def plot_meanCorr_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['mean_actv_corr'] for i in range(0, 12)]
    # unpaired_values = [layer_to_dictscores[i]['svcca_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    # rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('Corr')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    # label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [18]:
def plot_meanCorr_filt_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['mean_actv_corr_filt'] for i in range(0, 12)]

    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')

    ax.set_ylabel('Corr')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12)

    label_bars(rects1)
    plt.tight_layout()

    plt.show()

In [19]:
def plot_numFeats_afterFilt_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['num_feat_filt'] for i in range(0, 12)]

    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')

    ax.set_ylabel('Num Feats Kept')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12)

    label_bars(rects1)
    plt.tight_layout()

    plt.show()

In [20]:
# def plot_js_byLayer(layer_to_dictscores):
#     for key, sub_dict in layer_to_dictscores.items():
#         for sub_key, value in sub_dict.items():
#             sub_dict[sub_key] = round(value, 4)

#     layers = [f'L{i}' for i in range(0, 12)]
#     paired_values = [layer_to_dictscores[i]['1-1 jaccard_paired'] for i in range(0, 12)]
#     unpaired_values = [layer_to_dictscores[i]['1-1 jaccard_unpaired'] for i in range(0, 12)]

#     # Plotting configuration
#     x = np.arange(len(layers))  # label locations
#     width = 0.35  # width of the bars

#     # Increase figure size
#     fig, ax = plt.subplots(figsize=(12, 7))  # Slightly increased figure size
#     rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
#     rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

#     # Adding labels, title and custom x-axis tick labels
#     ax.set_ylabel('Jaccard NN')
#     ax.set_title('SAEs comparison by Pythia 70m MLP0 vs 160m MLP Layers')
#     ax.set_xticks(x)
#     ax.set_xticklabels(layers)
#     ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
#     ax.legend()

#     # Label bars with increased font size and different positioning for paired and unpaired
#     def label_bars(rects, is_paired):
#         for rect in rects:
#             height = rect.get_height()
#             label_height = height + 0.05 if is_paired else height + 0.01
#             ax.text(rect.get_x() + rect.get_width()/2., label_height,
#                     f'{height:.3f}',
#                     ha='center', va='bottom', fontsize=9)  # Increased font size to 12

#     label_bars(rects1, True)   # Paired bars
#     label_bars(rects2, False)  # Unpaired bars

#     # Adjust layout to prevent cutting off labels
#     plt.tight_layout()

#     # Increase y-axis limit to accommodate higher labels
#     ax.set_ylim(0, 1.1)  # Increased from 1 to 1.1

#     plt.show()

## interpret fns

In [21]:
def highest_activating_tokens(
    feature_acts,
    feature_idx: int,
    k: int = 10,  # num batch_seq samples
    batch_tokens=None
): # -> Tuple[Int[Tensor, "k 2"], Float[Tensor, "k"]]:
    '''
    Returns the indices & values for the highest-activating tokens in the given batch of data.
    '''
    batch_size, seq_len = batch_tokens.shape

    # Get the top k largest activations for only targeted feature
    # need to flatten (batch,seq) into batch*seq first because it's ANY batch_seq, even if in same batch or same pos
    flattened_feature_acts = feature_acts[:, :, feature_idx].reshape(-1)

    top_acts_values, top_acts_indices = flattened_feature_acts.topk(k)
    # top_acts_values should be 1D
    # top_acts_indices should be also be 1D. Now, turn it back to 2D
    # Convert the indices into (batch, seq) indices
    top_acts_batch = top_acts_indices // seq_len
    top_acts_seq = top_acts_indices % seq_len

    return torch.stack([top_acts_batch, top_acts_seq], dim=-1), top_acts_values

In [22]:
from rich import print as rprint
def display_top_sequences(top_acts_indices, top_acts_values, batch_tokens):
    s = ""
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        # s += f'{batch_idx}\n'
        s += f'batchID: {batch_idx}, '
        # Get the sequence as a string (with some padding on either side of our sequence)
        seq_start = max(seq_idx - 5, 0)
        seq_end = min(seq_idx + 5, batch_tokens.shape[1])
        seq = ""
        # Loop over the sequence, adding each token to the string (highlighting the token with the large activations)
        for i in range(seq_start, seq_end):
            # new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            new_str_token = tokenizer.decode([batch_tokens[batch_idx, i].item()]).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
            seq += new_str_token
        # Print the sequence, and the activation value
        s += f'Act = {value:.2f}, Seq = "{seq}"\n'

    rprint(s)

## search modB, get modA pair

In [23]:
def find_indices_with_keyword(fList, keyword):
    """
    Find all indices of fList which contain the keyword in the string at those indices.

    Args:
    fList (list of str): List of strings to search within.
    keyword (str): Keyword to search for within the strings of fList.

    Returns:
    list of int: List of indices where the keyword is found within the strings of fList.
    """
    index_list = []
    for index, string in enumerate(fList):
        split_list = string.split(',')
        no_space_list = [i.replace(' ', '').lower() for i in split_list]
        if keyword in no_space_list:
            index_list.append(index)
    return index_list

In [24]:
def get_values_from_indices(indices, values_list):
    """
    Get the values from values_list at the specified indices.

    Args:
    indices (list of int): List of indices to retrieve values from.
    values_list (list): List of values from which to retrieve the specified indices.

    Returns:
    list: List of values from values_list at the specified indices.
    """
    return [values_list[index] for index in indices]

## get actv fns

In [25]:
def get_weights_and_acts(name, cfg_dict, layer_id, outputs):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    hookpoint = "layers." + str(layer_id)
    decoder=True

    repo_path = Path(
                snapshot_download(
                    name,
                    allow_patterns=f"{hookpoint}/*" if hookpoint is not None else None,
                    # allow_patterns = None
                )
            )
    if hookpoint is not None:
        repo_path = repo_path / hookpoint
    path = Path(repo_path)
    # cfg_dict = {"expansion_factor": 32, "normalize_decoder": True, "num_latents": 32768, "k": 16, "d_in": d_in}
    d_in = cfg_dict.pop("d_in")
    cfg = SaeConfig(**cfg_dict)

    sae = Sae(d_in, cfg, device=device, decoder=decoder)

    load_model(
        model=sae,
        filename=str(path / "sae.safetensors"),
        device=str(device),
        strict=decoder,
    )

    weight_matrix_np = sae.W_dec.cpu().detach().numpy()

    with torch.inference_mode():
        reshaped_activations_A = sae.pre_acts(outputs.hidden_states[layer_id].to("cuda"))
        # orig = sae.pre_acts(outputs.hidden_states[layer_id].to("cuda"))

    first_dim_reshaped = reshaped_activations_A.shape[0] * reshaped_activations_A.shape[1]
    reshaped_activations_A = reshaped_activations_A.reshape(first_dim_reshaped, reshaped_activations_A.shape[-1]).cpu()

    # return weight_matrix_np, reshaped_activations_A, orig
    return weight_matrix_np, reshaped_activations_A

In [26]:
def count_zero_columns(tensor):
    # Check if all elements in each column are zero
    zero_columns = np.all(tensor == 0, axis=0)
    # Count True values in the zero_columns array
    zero_cols_indices = np.where(zero_columns)[0]
    return np.sum(zero_columns), zero_cols_indices

## run expm fns

In [27]:
def run_expm(layer_id, outputs, outputs_2):
    layer_to_dictscores = {}

    name = "EleutherAI/sae-pythia-70m-32k"
    cfg_dict = {"expansion_factor": 32, "normalize_decoder": True, "num_latents": 32768, "k": 16, "d_in": 512}
    weight_matrix_np, reshaped_activations_A = get_weights_and_acts(name, cfg_dict, layer_id, outputs)
    # zero_cols_count, zero_cols_indices = count_zero_columns(reshaped_activations_A.cpu().numpy())
    # print("Number of zero columns:", zero_cols_count) #, zero_cols_indices

    name = "EleutherAI/sae-pythia-160m-32k"
    for layerID_2 in range(0, 12): # 0, 12
        dictscores = {}

        # redef
        cfg_dict = {"expansion_factor": 32, "normalize_decoder": True, "num_latents": 32768, "k": 16, "d_in": 768}
        weight_matrix_2, reshaped_activations_B = get_weights_and_acts(name, cfg_dict, layerID_2, outputs_2)
        # zero_cols_count, zero_cols_indices = count_zero_columns(reshaped_activations_B.cpu().numpy())

        # hookpoint = "layers." + str(layerID_2)
        # decoder=True

        # repo_path = Path(
        #             snapshot_download(
        #                 name,
        #                 allow_patterns=f"{hookpoint}/*" if hookpoint is not None else None,
        #                 # allow_patterns = None
        #             )
        #         )
        # if hookpoint is not None:
        #     repo_path = repo_path / hookpoint
        # path = Path(repo_path)
        # cfg_dict = {"expansion_factor": 32, "normalize_decoder": True, "num_latents": 32768, "k": 16, "d_in": 768}
        # d_in = cfg_dict.pop("d_in")
        # cfg = SaeConfig(**cfg_dict)

        # sae_2 = Sae(d_in, cfg, device=device, decoder=decoder)

        # load_model(
        #     model=sae_2,
        #     filename=str(path / "sae.safetensors"),
        #     device=str(device),
        #     strict=decoder,
        # )

        # weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

        # with torch.inference_mode():
        #     reshaped_activations_B = sae_2.pre_acts(outputs_2.hidden_states[layerID_2].to("cuda"))

        # first_dim_reshaped = reshaped_activations_B.shape[0] * reshaped_activations_B.shape[1]
        # reshaped_activations_B = reshaped_activations_B.reshape(first_dim_reshaped, reshaped_activations_B.shape[-1]).cpu()

        """
        `batched_correlation(reshaped_activations_B, reshaped_activations_A)`:
        highest_correlations_indices_AB contains modA's feats as inds, and modB's feats as vals.
        Use the list with smaller number of features (cols) as the second arg
        """
        highest_correlations_indices_AB, highest_correlations_values_AB = batched_correlation(reshaped_activations_A, reshaped_activations_B)
        highest_correlations_indices_AB = highest_correlations_indices_AB.detach().cpu().numpy()
        highest_correlations_values_AB = highest_correlations_values_AB.detach().cpu().numpy()

        num_unq_pairs = len(list(set(highest_correlations_indices_AB)))
        print("% unique: ", num_unq_pairs / len(highest_correlations_indices_AB))

        dictscores["mean_actv_corr"] = sum(highest_correlations_values_AB) / len(highest_correlations_values_AB)

        ###########
        # filter

        sorted_feat_counts = Counter(highest_correlations_indices_AB).most_common()
        # kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count <= 100]
        kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count <= 1]

        filt_corr_ind_A = []
        filt_corr_ind_B = []
        seen = set()
        for ind_B, ind_A in enumerate(highest_correlations_indices_AB):
            if ind_A in kept_modA_feats:
                filt_corr_ind_A.append(ind_A)
                filt_corr_ind_B.append(ind_B)
            elif ind_A not in seen:  # only keep one if it's over count X
                seen.add(ind_A)
                filt_corr_ind_A.append(ind_A)
                filt_corr_ind_B.append(ind_B)
        # num_unq_pairs = len(list(set(filt_corr_ind_A)))
        # print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
        # print("num feats after filt: ", len(filt_corr_ind_A))

        new_highest_correlations_indices_A = []
        new_highest_correlations_indices_B = []
        new_highest_correlations_values = []

        for ind_A, ind_B in zip(filt_corr_ind_A, filt_corr_ind_B):
            val = highest_correlations_values_AB[ind_B]
            if val > 0:
                new_highest_correlations_indices_A.append(ind_A)
                new_highest_correlations_indices_B.append(ind_B)
                new_highest_correlations_values.append(val)

        # print("% unique after rmv 0s: ", num_unq_pairs / len(new_highest_correlations_indices_A))
        # print("num feats after rmv 0s: ", len(new_highest_correlations_indices_A))
        dictscores["num_feat_kept"] = len(new_highest_correlations_indices_A)

        dictscores["mean_actv_corr_filt"] = sum(new_highest_correlations_values) / len(new_highest_correlations_values)

        ###########
        # sim tests

        # # num_feats = len(filt_corr_ind_A)
        num_feats = len(new_highest_correlations_indices_A)

        # # dictscores["svcca_paired"] = svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")
        # dictscores["svcca_paired"] = svcca(weight_matrix_np[new_highest_correlations_indices_A], weight_matrix_2[new_highest_correlations_indices_B], "nd")

        # dictscores["svcca_unpaired"] = score_rand(weight_matrix_np, weight_matrix_2, num_feats,
        #                                           svcca, shapereq_bool=True)

        dictscores["rsa_paired"] = representational_similarity_analysis(weight_matrix_np[new_highest_correlations_indices_A], weight_matrix_2[new_highest_correlations_indices_B], "nd")
        dictscores["rsa_unpaired"] = score_rand(weight_matrix_np, weight_matrix_2, num_feats,
                                                  representational_similarity_analysis, shapereq_bool=True)

        print("Layer: " + str(layerID_2))
        for key, value in dictscores.items():
            print(key + ": " + str(value))
        print("\n")

        layer_to_dictscores[layerID_2] = dictscores
    return layer_to_dictscores

# load data

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
from datasets import load_dataset
# dataset = load_dataset("roneneldan/TinyStories", split="train", streaming=True)
dataset = load_dataset("Skylion007/openwebtext", split="train", streaming=True)

openwebtext.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [30]:
batch_size = 100
maxseqlen = 300

def get_next_batch(dataset_iter, batch_size=100):
    batch = []
    for _ in range(batch_size):
        try:
            sample = next(dataset_iter)
            batch.append(sample['text'])
        except StopIteration:
            break
    return batch

dataset_iter = iter(dataset)
batch = get_next_batch(dataset_iter, batch_size)

# Tokenize the batch
inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=maxseqlen)

# load models

In [31]:
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")
model_2 = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m")

config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

## get LLM actvs

In [32]:
with torch.inference_mode():
    outputs = model(**inputs, output_hidden_states=True)
    outputs_2 = model_2(**inputs, output_hidden_states=True)

# MLP2 v L2

## sae actvs

In [33]:
layer_id = 2
name = "EleutherAI/sae-pythia-70m-32k"
hookpoint = "layers." + str(layer_id)
sae = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

with torch.inference_mode():
    outputs = model(**inputs, output_hidden_states=True)
    feature_acts_A = sae.pre_acts(outputs.hidden_states[layer_id].to('cuda'))

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

layers.2/cfg.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sae.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

In [34]:
first_dim_reshaped = feature_acts_A.shape[0] * feature_acts_A.shape[1]
reshaped_activations_A = feature_acts_A.reshape(first_dim_reshaped, feature_acts_A.shape[-1]).cpu()

In [35]:
weight_matrix_np = sae.W_dec.cpu().detach().numpy()

In [36]:
layer_id_2 = 2
name = "EleutherAI/sae-pythia-160m-32k"
hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

layers.2/cfg.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [37]:
with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

In [38]:
first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

In [39]:
weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

## corr

In [40]:
"""
`batched_correlation(reshaped_activations_B, reshaped_activations_A)` : highest_correlations_indices_AB contains modA's feats as inds, and modB's feats as vals. Use the list with smaller number of features (cols) as the second arg
(reshaped_activations_A, reshaped_activations_B): modB is inds, modA is vals of highest_correlations_indices_AB
"""
highest_correlations_indices_AB, highest_correlations_values_AB = batched_correlation(reshaped_activations_A, reshaped_activations_B)
highest_correlations_indices_AB = highest_correlations_indices_AB.detach().cpu().numpy()
highest_correlations_values_AB = highest_correlations_values_AB.detach().cpu().numpy()

num_unq_pairs = len(list(set(highest_correlations_indices_AB)))
print("% unique: ", num_unq_pairs / len(highest_correlations_indices_AB))

# dictscores["mean_actv_corr"] =
sum(highest_correlations_values_AB) / len(highest_correlations_values_AB)

% unique:  0.23333740234375


0.6951166202568402

In [41]:
sorted_feat_counts = Counter(highest_correlations_indices_AB).most_common()
kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count == 1]
len(kept_modA_feats)

2663

## save ds for features

In [ ]:
inputs["input_ids"][0, 0]

tensor(11688)

In [ ]:
tokenizer.decode(inputs["input_ids"][0, 0])

'Port'

In [ ]:
def store_top_sequences_asLst(top_acts_indices, top_acts_values, batch_tokens):
    # s = ""
    feat_samps = []
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        # s += f'batchID: {batch_idx}, '
        seq_start = max(seq_idx - 5, 0)
        seq_end = min(seq_idx + 5, batch_tokens.shape[1])
        seq = ""
        for i in range(seq_start, seq_end):
            new_str_token = tokenizer.decode(batch_tokens[batch_idx, i]).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            # new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
            seq += new_str_token
        # s += f'Act = {value:.2f}, Seq = "{seq}"\n'
        feat_samps.append(seq)
    return feat_samps

In [ ]:
# store feature : lst of top strs
feat_snip_dict = {}
samp_m = 5

for feature_idx in range(feature_acts_B.shape[-1]):
    if feature_idx % 5000 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_B, feature_idx, samp_m, batch_tokens=inputs["input_ids"])
    feat_snip_dict[feature_idx] = store_top_sequences_asLst(ds_top_acts_indices, ds_top_acts_values, inputs["input_ids"])

Feature:  0
Feature:  5000
Feature:  10000
Feature:  15000
Feature:  20000
Feature:  25000
Feature:  30000


In [ ]:
feature_top_samps_lst = [{"feature": feature, "strings": strings} for feature, strings in feat_snip_dict.items()]

In [ ]:
import re

def extract_tagged_word(s):
    # Define the regex pattern to match the tagged word
    pattern = r'\[bold u dark_orange\](.*?)\[/\]'

    # Search for the pattern in the string
    match = re.search(pattern, s)

    # If a match is found, return the captured group (the word inside the tags)
    if match:
        return match.group(1)
    else:
        return None

In [ ]:
fList_model_B = []
for feat_dict in feature_top_samps_lst:
    out_str = ''
    for text in feat_dict['strings']:
        result = extract_tagged_word(text)
        out_str += result + ', '
    fList_model_B.append(out_str)

## save spliced labels

In [ ]:
from google.colab import files

In [ ]:
with open('feat_snip_dict_B.pkl', 'wb') as f:
    pickle.dump(feat_snip_dict, f)
files.download('feat_snip_dict_B.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# with open('fList_model_A.pkl', 'wb') as f:
#     pickle.dump(fList_model_A, f)
# files.download('fList_model_A.pkl')

with open('fList_model_B.pkl', 'wb') as f:
    pickle.dump(fList_model_B, f)
files.download('fList_model_B.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
import pickle
with open('fList_model_A.pkl', 'rb') as f:
    fList_model_A = pickle.load(f)
with open('fList_model_B.pkl', 'rb') as f:
    fList_model_B = pickle.load(f)

# match by semantic subspace- appr 1

APPR1: match by most corr pair

In [ ]:
samp_m = 5

## she

In [ ]:
keyword = "she"
modB_feats = find_indices_with_keyword(fList_model_B, keyword)
modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
X_subset = weight_matrix_np[modA_feats, :]
Y_subset = weight_matrix_2[modB_feats, :]

In [ ]:
for fID in range(len(modB_feats[:5])):
    print('Corr: ' + str(highest_correlations_values_AB[modB_feats[fID]]) + '\n')
    feature_idx_A = modA_feats[fID]
    print('Model A Feature: ', feature_idx_A)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_A, feature_idx_A, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

    feature_idx_B = modB_feats[fID]
    print('Model B Feature: ', feature_idx_B)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_B, feature_idx_B, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

    print('-'*50)

Corr: 0.58918166

Model A Feature:  23687


batchID: 0, Act = 5.28, Seq = " the team.\n\nHe said it was a"
batchID: 63, Act = 5.27, Seq = " aged 60.\n\nHe had a progressive lung"
batchID: 96, Act = 5.27, Seq = ",” Quinn said. “He was on the plan"
batchID: 98, Act = 5.26, Seq = " stand-up days. He alleges that this was"
batchID: 6, Act = 5.25, Seq = " from credit card machines. He says he plans to"

Model B Feature:  653


batchID: 29, Act = 1.56, Seq = ". She probably, maybe she wasn’t allowed"
batchID: 96, Act = 1.49, Seq = " and then some team. He worked some team today"
batchID: 72, Act = 1.48, Seq = "He was a few blocks"
batchID: 89, Act = 1.45, Seq = "’re headed, who they’re there to"
batchID: 2, Act = 1.39, Seq = " He simply lamented that he hadn't done a"

--------------------------------------------------
Corr: 0.75935507

Model A Feature:  23687


...

--------------------------------------------------
Corr: 0.36430278

Model A Feature:  23687


batchID: 0, Act = 5.28, Seq = " the team.\n\nHe said it was a"
batchID: 63, Act = 5.27, Seq = " aged 60.\n\nHe had a progressive lung"
batchID: 96, Act = 5.27, Seq = ",” Quinn said. “He was on the plan"
batchID: 98, Act = 5.26, Seq = " stand-up days. He alleges that this was"
batchID: 6, Act = 5.25, Seq = " from credit card machines. He says he plans to"

Model B Feature:  11962


batchID: 30, Act = 0.47, Seq = " Facebook and Google, where she also worked), a"
batchID: 81, Act = 0.34, Seq = " stupidity as badly as they did down at the"
batchID: 46, Act = 0.32, Seq = " Paws Den, where he has to adapt to"
batchID: 96, Act = 0.30, Seq = "ley said after the morning practice. “I’"
batchID: 33, Act = 0.30, Seq = " predictable. The only changes they’ve made so"

--------------------------------------------------


In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)

In [ ]:
len(new_vals)

15

In [ ]:
sum(new_vals) / len(new_vals)

0.5829166144132614

In [ ]:
print(len(list(set(modB_feats))))

22


In [ ]:
print(len(list(set(modA_feats))))

15


In [ ]:
svcca(X_subset, Y_subset, "nd")

0.5218224922427003

In [ ]:
num_feats = len(list(set(modA_feats)))
score_rand(100, weight_matrix_np, weight_matrix_2, num_feats,
                                          svcca, shapereq_bool=True)

0.2185287201231882

In [ ]:
representational_similarity_analysis(X_subset, Y_subset, "nd")

0.12556193194351814

In [ ]:
num_feats = len(list(set(modA_feats)))
score_rand(100, weight_matrix_np, weight_matrix_2, num_feats,
                                          representational_similarity_analysis, shapereq_bool=True)

0.010969310276008393

### 1-1 only

In [ ]:
filt_corr_ind_A = []
filt_corr_ind_B = []
seen = set()
# for ind_B, ind_A in enumerate(highest_correlations_indices_AB):
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A in kept_modA_feats:
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
    elif ind_A not in seen:  # only keep one if it's over count X
        seen.add(ind_A)
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
num_unq_pairs = len(list(set(filt_corr_ind_A)))
print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
print("num feats after filt: ", len(filt_corr_ind_A))

% unique:  1.0
num feats after filt:  15


In [ ]:
svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")

0.1846519210823571

## he

In [ ]:
keyword = "he"
modB_feats = find_indices_with_keyword(fList_model_B, keyword)
modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
X_subset = weight_matrix_np[modA_feats, :]
Y_subset = weight_matrix_2[modB_feats, :]

In [ ]:
for fID in range(len(modB_feats[:5])):
    print('Corr: ' + str(highest_correlations_values_AB[modB_feats[fID]]) + '\n')
    feature_idx_A = modA_feats[fID]
    print('Model A Feature: ', feature_idx_A)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_A, feature_idx_A, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

    feature_idx_B = modB_feats[fID]
    print('Model B Feature: ', feature_idx_B)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_B, feature_idx_B, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

    print('-'*50)

Corr: 0.15237112

Model A Feature:  25476


batchID: 36, Act = 0.46, Seq = "Whenever something momentous happens"
batchID: 3, Act = 0.45, Seq = " those musical entities that transcends language. It’"
batchID: 36, Act = 0.43, Seq = "Whenever something momentous happens in"
batchID: 50, Act = 0.42, Seq = " cycle, gun rights organizations spent 34 times more on"
batchID: 72, Act = 0.41, Seq = " a Hamilton Police officer who seemed not to realize who"

Model B Feature:  501


batchID: 72, Act = 1.29, Seq = " seemed not to realize who he was.\n\n"
batchID: 39, Act = 1.05, Seq = " Wiggins, who by winning a gold medal"
batchID: 64, Act = 1.04, Seq = "ama Rao, who like Gabbard,"
batchID: 72, Act = 0.97, Seq = " an impact that people who oppose carding and street"
batchID: 85, Act = 0.93, Seq = "a, 23, who drove down to the station"

--------------------------------------------------
Corr: 0.58918166

Model A Feature:  23687


...

--------------------------------------------------
Corr: 0.460781

Model A Feature:  19938


batchID: 5, Act = 0.07, Seq = " censorship, “blasphemy laws” and"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince, Haiti"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince, Haiti ("
batchID: 0, Act = 0.00, Seq = "Port-au-Prince,"

Model B Feature:  1446


batchID: 1, Act = 0.53, Seq = " and her rival, Bernie Sanders, spoke about the"
batchID: 5, Act = 0.50, Seq = " censorship, “blasphemy laws” and"
batchID: 1, Act = 0.47, Seq = " whether rival Sen. Bernie Sanders of Vermont would seek"
batchID: 79, Act = 0.31, Seq = "The Catholic Archdiocese of Kansas City has"
batchID: 1, Act = 0.27, Seq = " Post)\n\nJeff Weaver, Sanders’s"

--------------------------------------------------


In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)

In [ ]:
len(new_vals)

56

In [ ]:
sum(new_vals) / len(new_vals)

0.6245164323065963

In [ ]:
print(len(list(set(modB_feats))))

82


In [ ]:
print(len(list(set(modA_feats))))

56


In [ ]:
svcca(X_subset, Y_subset, "nd")

0.20732837892303863

In [ ]:
num_feats = len(list(set(modA_feats)))
score_rand(100, weight_matrix_np, weight_matrix_2, num_feats,
                                          svcca, shapereq_bool=True)

0.10339815164308169

In [ ]:
representational_similarity_analysis(X_subset, Y_subset, "nd")

0.07612498486714694

In [ ]:
num_feats = len(list(set(modA_feats)))
score_rand(100, weight_matrix_np, weight_matrix_2, num_feats,
                                          representational_similarity_analysis, shapereq_bool=True)

0.0018538637989886056

## rand subset of 20 corr feats

In [ ]:
modB_feats = list(range(20))
modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
X_subset = weight_matrix_np[modA_feats, :]
Y_subset = weight_matrix_2[modB_feats, :]

In [ ]:
for fID in range(len(modB_feats[:5])):
    print('Corr: ' + str(highest_correlations_values_AB[modB_feats[fID]]) + '\n')
    feature_idx_A = modA_feats[fID]
    print('Model A Feature: ', feature_idx_A)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_A, feature_idx_A, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

    feature_idx_B = modB_feats[fID]
    print('Model B Feature: ', feature_idx_B)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_B, feature_idx_B, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

    print('-'*50)

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)

In [ ]:
sum(new_vals) / len(new_vals)

0.6549430556595326

In [ ]:
print(len(list(set(modB_feats))))

20


In [ ]:
print(len(list(set(modA_feats))))

20


In [ ]:
svcca(X_subset, Y_subset, "nd")

0.05127905844016922

In [ ]:
num_feats = len(list(set(modA_feats)))
score_rand(100, weight_matrix_np, weight_matrix_2, num_feats,
                                          svcca, shapereq_bool=True)

0.1745702333360795

In [ ]:
representational_similarity_analysis(X_subset, Y_subset, "nd")

0.09046263127277182

In [ ]:
num_feats = len(list(set(modA_feats)))
score_rand(100, weight_matrix_np, weight_matrix_2, num_feats,
                                          representational_similarity_analysis, shapereq_bool=True)

0.008302977141539755

### shuffle subset pairing

In [ ]:
shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.17493517093743535

## rand subset of 1000 corr feats

In [ ]:
modB_feats = list(range(1000))
modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
X_subset = weight_matrix_np[modA_feats, :]
Y_subset = weight_matrix_2[modB_feats, :]

In [ ]:
for fID in range(len(modB_feats[:5])):
    print('Corr: ' + str(highest_correlations_values_AB[modB_feats[fID]]) + '\n')
    feature_idx_A = modA_feats[fID]
    print('Model A Feature: ', feature_idx_A)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_A, feature_idx_A, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

    feature_idx_B = modB_feats[fID]
    print('Model B Feature: ', feature_idx_B)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_B, feature_idx_B, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

    print('-'*50)

Corr: 0.2252538

Model A Feature:  3391


batchID: 32, Act = 2.94, Seq = " II in 2015. Adams joined the New York Red"
batchID: 32, Act = 2.86, Seq = ".\n\nAdams joined the U.S"
batchID: 53, Act = 2.43, Seq = "\n\nMeanwhile, protesters lifted blockade from various places"
batchID: 63, Act = 2.41, Seq = "s and 1980s he hosted shows including Chegg"
batchID: 3, Act = 2.37, Seq = ", NJ. As I waited in line to enter"

Model B Feature:  0


batchID: 45, Act = 0.67, Seq = " ACC piece.) Odds change as more evidence becomes"
batchID: 28, Act = 0.66, Seq = "As President Barack Obama begins his second term,"
batchID: 85, Act = 0.65, Seq = " why friends shouldn’t let friends drive drunk."
batchID: 44, Act = 0.64, Seq = "emen. Poekel understands impatient ambition — he"
batchID: 6, Act = 0.61, Seq = "× Some Seattle businesses closed for ‘A Day"

--------------------------------------------------
Corr: 0.768062

Model A Feature:  2604


...

--------------------------------------------------
Corr: 0.8589737

Model A Feature:  16501


batchID: 95, Act = 3.34, Seq = " player can have a lull in his game,"
batchID: 32, Act = 1.73, Seq = " THE NEW YORK RED BULLS EMAIL"
batchID: 84, Act = 1.46, Seq = " equipment, according to Liz Bullard, who helped"
batchID: 84, Act = 1.35, Seq = " kids move in," writes Bullard.\n\n"
batchID: 14, Act = 0.12, Seq = "Get cool in-game extras with ami"

Model B Feature:  4


batchID: 95, Act = 5.14, Seq = " player can have a lull in his game,"
batchID: 32, Act = 1.28, Seq = " THE NEW YORK RED BULLS EMAIL"
batchID: 50, Act = 0.48, Seq = "aved parents and lawmakers reissue a call for more"
batchID: 96, Act = 0.43, Seq = " are hoping their first-round pick, who has"
batchID: 64, Act = 0.28, Seq = " of Diwali, reflecting on the significance of"

--------------------------------------------------


In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)

In [ ]:
len(new_vals)

700

In [ ]:
sum(new_vals) / len(new_vals)

0.6873051235292639

In [ ]:
print(len(list(set(modB_feats))))

1000


In [ ]:
print(len(list(set(modA_feats))))

700


In [ ]:
svcca(X_subset, Y_subset, "nd")

9.289818936991208e-05

In [ ]:
num_feats = len(list(set(modA_feats)))
score_rand(10, weight_matrix_np, weight_matrix_2, num_feats,
                                          svcca, shapereq_bool=True)

0.036708355590066556

In [ ]:
representational_similarity_analysis(X_subset, Y_subset, "nd")

0.03667503611346406

In [ ]:
num_feats = len(list(set(modA_feats)))
score_rand(10, weight_matrix_np, weight_matrix_2, num_feats,
                                          representational_similarity_analysis, shapereq_bool=True)

-0.0005436258460439472

### shuffle subset pairing

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.027769560837738645

### take more rand PAIRED subsets

In [ ]:
scores = []
num_feats = 1000
for i in range(10):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = svcca(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.08116187993215342

In [ ]:
scores

[0.1698141170043281,
 0.043710113142629174,
 0.17825191273573127,
 0.14282031790148828,
 0.07048045738465511,
 0.0163270197632972,
 0.08494968523083468,
 0.02146806622039517,
 0.0329009160947603,
 0.05089619384341473]

In [ ]:
scores[-1]

0.05089619384341473

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6650007739348753

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.028959496616092435

## combos of numerics

In [ ]:
mixed_modA_feats = []
mixed_modB_feats = []
keywords = [
    "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
    "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen", "twenty",
    "hundred", "thousand", "million", "billion", "trillion",
    "integer", "fraction", "decimal", "percentage", "ratio",
    "numeral", "digit", "prime", "even", "odd",
    "sum", "difference", "product", "factor", "multiple",
    "total", "count", "measure", "dozen", "score", "unit"
]

for kw in keywords:
    modB_feats = find_indices_with_keyword(fList_model_B, kw)
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    mixed_modA_feats.extend(modA_feats)
    mixed_modB_feats.extend(modB_feats)

print( len(list(set(mixed_modA_feats))) )
len(list(set(mixed_modB_feats)))

277


417

In [ ]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.4705058013121089

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.05939620992899267

### 1-1 only

In [ ]:
filt_corr_ind_A = []
filt_corr_ind_B = []
seen = set()
for ind_A, ind_B in zip(mixed_modA_feats, mixed_modB_feats):
    if ind_A in kept_modA_feats:
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
    elif ind_A not in seen:  # only keep one if it's over count X
        seen.add(ind_A)
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
num_unq_pairs = len(list(set(filt_corr_ind_A)))
print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
print("num feats after filt: ", len(filt_corr_ind_A))

% unique:  0.9421768707482994
num feats after filt:  294


In [ ]:
svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")

0.2840058404315349

### take more rand PAIRED subsets

In [ ]:
num_feats = len(list(set(mixed_modA_feats)))
scores = []
for i in range(1000):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = svcca(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.10861926780843162

In [ ]:
p_value = np.mean(np.array(scores) >= paired_svcca)
p_value

0.0

In [ ]:
import statistics
variance_value = statistics.variance(scores)
print(f"Variance: {variance_value}")

Variance: 0.007021783344632679


In [ ]:
import math
std_dev = statistics.stdev(scores)
n = len(scores)
standard_error = std_dev / math.sqrt(n)
print(f"Standard Error: {standard_error}")

Standard Error: 0.0026498647785561965


In [ ]:
scores[-1]

0.30799825512215495

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6493480509103731

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.059253895791384216

### rsa

In [ ]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

In [ ]:
paired_score = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_score

0.05066869736268442

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          representational_similarity_analysis, shapereq_bool=True)

0.0005058621912115877

In [ ]:
num_feats = len(list(set(mixed_modA_feats)))
scores = []
for i in range(1000):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = representational_similarity_analysis(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.0353266686531004

In [ ]:
p_value = np.mean(np.array(scores) >= paired_score)
p_value

0.052

In [ ]:
import statistics
variance_value = statistics.variance(scores)
print(f"Variance: {variance_value}")

Variance: 8.797439334983352e-05


In [ ]:
import math
std_dev = statistics.stdev(scores)
n = len(scores)
standard_error = std_dev / math.sqrt(n)
print(f"Standard Error: {standard_error}")

Standard Error: 0.0002966047763435942


In [ ]:
scores[-1]

0.029871989637636712

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6574529204844383

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.03765311279579574

## combos of people-related

In [ ]:
mixed_modA_feats = []
mixed_modB_feats = []
keywords = [
    "man", "girl", "boy", "kid", "dad", "mom", "son", "sis", "bro",
    "pal", "mate", "boss", "chief", "cop", "guide", "priest", "king",
    "queen", "duke", "lord", "friend", "judge", "clerk", "coach", "team",
    "crew", "staff", "nurse", "doc", "vet", "cook", "maid", "clown",
    "star", "clan", "host", "guest", "peer", "guard", "boss", "spy",
    "fool", "punk", "nerd", "jock", "chief", "folk", "crowd"
]

for kw in keywords:
    modB_feats = find_indices_with_keyword(fList_model_B, kw)
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    mixed_modA_feats.extend(modA_feats)
    mixed_modB_feats.extend(modB_feats)

print( len(list(set(mixed_modA_feats))) )
len(list(set(mixed_modB_feats)))

283


400

In [ ]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.35236953435122914

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.029819445561711046

### 1-1 only

In [ ]:
filt_corr_ind_A = []
filt_corr_ind_B = []
seen = set()
for ind_A, ind_B in zip(mixed_modA_feats, mixed_modB_feats):
    if ind_A in kept_modA_feats:
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
    elif ind_A not in seen:  # only keep one if it's over count X
        seen.add(ind_A)
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
num_unq_pairs = len(list(set(filt_corr_ind_A)))
print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
print("num feats after filt: ", len(filt_corr_ind_A))

% unique:  0.9929824561403509
num feats after filt:  285


In [ ]:
svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")

0.34276975330517323

### take more rand PAIRED subsets

In [ ]:
num_feats = len(list(set(mixed_modA_feats)))
scores = []
for i in range(100):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = svcca(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.10080474849583716

In [ ]:
p_value = np.mean(np.array(scores) >= paired_svcca)
p_value

0.0

In [ ]:
import statistics
variance_value = statistics.variance(scores)
print(f"Variance: {variance_value}")

Variance: 0.00698736865985887


In [ ]:
import math
std_dev = statistics.stdev(scores)
n = len(scores)
standard_error = std_dev / math.sqrt(n)
print(f"Standard Error: {standard_error}")

Standard Error: 0.008359048187358935


In [ ]:
scores[-1]

0.0894112828770815

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6669796037240925

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.03437358096425061

### rsa

In [ ]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

In [ ]:
paired_score = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_score

0.058729009030208605

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          representational_similarity_analysis, shapereq_bool=True)

0.0030277978078067313

In [ ]:
num_feats = len(list(set(mixed_modA_feats)))
scores = []
for i in range(1000):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = representational_similarity_analysis(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.035279482133332395

In [ ]:
p_value = np.mean(np.array(scores) >= paired_score)
p_value

0.013

In [ ]:
import statistics
variance_value = statistics.variance(scores)
print(f"Variance: {variance_value}")

Variance: 8.708523698729906e-05


In [ ]:
import math
std_dev = statistics.stdev(scores)
n = len(scores)
standard_error = std_dev / math.sqrt(n)
print(f"Standard Error: {standard_error}")

Standard Error: 0.00029510207892744346


In [ ]:
scores[-1]

0.03843016203404345

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6929317576756616

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.041581790418365044

## combos of nature-related

In [ ]:
mixed_modA_feats = []
mixed_modB_feats = []
keywords = [
    "tree", "grass", "bush", "plant", "stone", "rock", "cliff", "hill",
    "dirt", "sand", "mud", "wind", "storm", "rain", "cloud", "sun",
    "moon", "star", "leaf", "branch", "twig", "root", "bark", "seed",
    "wave", "tide", "lake", "pond", "creek", "sea", "wood", "field",
    "shore", "snow", "ice", "flame", "fire", "fog", "dew", "hail",
    "sky", "earth", "glade", "cave", "peak", "ridge", "dust", "air",
    "mist", "heat"
]

for kw in keywords:
    modB_feats = find_indices_with_keyword(fList_model_B, kw)
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    mixed_modA_feats.extend(modA_feats)
    mixed_modB_feats.extend(modB_feats)

print( len(list(set(mixed_modA_feats))) )
len(list(set(mixed_modB_feats)))

213


304

In [ ]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.3659125854383825

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.038049738715207934

### 1-1 only

In [ ]:
filt_corr_ind_A = []
filt_corr_ind_B = []
seen = set()
for ind_A, ind_B in zip(mixed_modA_feats, mixed_modB_feats):
    if ind_A in kept_modA_feats:
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
    elif ind_A not in seen:  # only keep one if it's over count X
        seen.add(ind_A)
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
num_unq_pairs = len(list(set(filt_corr_ind_A)))
print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
print("num feats after filt: ", len(filt_corr_ind_A))

% unique:  0.9953271028037384
num feats after filt:  214


In [ ]:
svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")

0.30383355481088264

### take more rand PAIRED subsets

In [ ]:
num_feats = len(list(set(mixed_modA_feats)))
scores = []
for i in range(100):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = svcca(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.11030848178395548

In [ ]:
p_value = np.mean(np.array(scores) >= paired_svcca)
p_value

0.0

In [ ]:
import statistics
variance_value = statistics.variance(scores)
print(f"Variance: {variance_value}")

Variance: 0.00568809847418735


In [ ]:
import math
std_dev = statistics.stdev(scores)
n = len(scores)
standard_error = std_dev / math.sqrt(n)
print(f"Standard Error: {standard_error}")

Standard Error: 0.007541948338584235


In [ ]:
scores[-1]

0.19468864479706188

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6564088905496257

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.06076239735700186

### rsa

In [ ]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

In [ ]:
paired_score = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_score

0.0703441429804715

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          representational_similarity_analysis, shapereq_bool=True)

0.0016477066422209846

In [ ]:
num_feats = len(list(set(mixed_modA_feats)))
scores = []
for i in range(100):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = representational_similarity_analysis(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.035242983088728225

In [ ]:
p_value = np.mean(np.array(scores) >= paired_score)
p_value

0.01

In [ ]:
import statistics
variance_value = statistics.variance(scores)
print(f"Variance: {variance_value}")

Variance: 0.00018379345228299978


In [ ]:
import math
std_dev = statistics.stdev(scores)
n = len(scores)
standard_error = std_dev / math.sqrt(n)
print(f"Standard Error: {standard_error}")

Standard Error: 0.0013557044378587828


In [ ]:
scores[-1]

0.05480374872258104

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6636336269392364

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.05558326850380363

## combos of science-related

In [ ]:
mixed_modA_feats = []
mixed_modB_feats = []
keywords = [
    "cell", "gene", "nerve", "brain", "blood", "bone", "heart", "lung",
    "star", "space", "light", "mass", "force", "wave", "speed", "sound",
    "time", "power", "heat", "cold", "charge", "spark", "flame", "bond",
    "quark", "atom", "ion", "gas", "wind", "ice", "plant", "rock",
    "probe", "test", "fact", "proof", "code", "law", "rule", "graph",
    "scale", "scope", "lens", "ray", "line", "chart", "flux", "phase",
    "shock", "pulse"
]

for kw in keywords:
    modB_feats = find_indices_with_keyword(fList_model_B, kw)
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    mixed_modA_feats.extend(modA_feats)
    mixed_modB_feats.extend(modB_feats)

print( len(list(set(mixed_modA_feats))) )
len(list(set(mixed_modB_feats)))

555


879

In [ ]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.32419862600866334

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.027145340336945722

### 1-1 only

In [ ]:
filt_corr_ind_A = []
filt_corr_ind_B = []
seen = set()
for ind_A, ind_B in zip(mixed_modA_feats, mixed_modB_feats):
    if ind_A in kept_modA_feats:
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
    elif ind_A not in seen:  # only keep one if it's over count X
        seen.add(ind_A)
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
num_unq_pairs = len(list(set(filt_corr_ind_A)))
print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
print("num feats after filt: ", len(filt_corr_ind_A))

% unique:  0.9982014388489209
num feats after filt:  556


In [ ]:
svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")

0.3126951472322523

### take more rand PAIRED subsets

In [ ]:
num_feats = len(list(set(mixed_modA_feats)))
scores = []
for i in range(100):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = svcca(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.09982650635791337

In [ ]:
p_value = np.mean(np.array(scores) >= paired_svcca)
p_value

0.0

In [ ]:
import statistics
variance_value = statistics.variance(scores)
print(f"Variance: {variance_value}")

Variance: 0.00667674366073616


In [ ]:
import math
std_dev = statistics.stdev(scores)
n = len(scores)
standard_error = std_dev / math.sqrt(n)
print(f"Standard Error: {standard_error}")

Standard Error: 0.008171134352546261


In [ ]:
scores[-1]

0.25365830000715966

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6675765203510963

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.021991843248726363

### rsa

In [ ]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

In [ ]:
paired_score = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_score

0.04714141744149278

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          representational_similarity_analysis, shapereq_bool=True)

-0.0008189849485021941

In [ ]:
num_feats = len(list(set(mixed_modA_feats)))
scores = []
for i in range(100):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = representational_similarity_analysis(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.03539399797445168

In [ ]:
p_value = np.mean(np.array(scores) >= paired_score)
p_value

0.07

In [ ]:
import statistics
variance_value = statistics.variance(scores)
print(f"Variance: {variance_value}")

Variance: 4.700747568487273e-05


In [ ]:
import math
std_dev = statistics.stdev(scores)
n = len(scores)
standard_error = std_dev / math.sqrt(n)
print(f"Standard Error: {standard_error}")

Standard Error: 0.0006856199799077673


In [ ]:
scores[-1]

0.048139493298109445

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6600648197843173

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.039418307584225654

## combos of animal-related

In [ ]:
mixed_modA_feats = []
mixed_modB_feats = []
keywords = [
    "dog", "cat", "rat", "bat", "pig", "cow", "fox", "wolf", "ram", "eel",
    "ant", "bee", "bug", "cub", "kit", "fawn", "calf", "colt", "foal",
    "hen", "duck", "goat", "bird", "crow", "fish", "frog", "deer", "worm",
    "moth", "gnat", "clam", "crab", "shrimp", "whale", "shark", "squid",
    "pup", "joey", "owl", "hare", "seal", "mule", "toad", "swan", "sow",
    "bull", "stag", "buck", "boar", "kite"
]


for kw in keywords:
    modB_feats = find_indices_with_keyword(fList_model_B, kw)
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    mixed_modA_feats.extend(modA_feats)
    mixed_modB_feats.extend(modB_feats)

print( len(list(set(mixed_modA_feats))) )
len(list(set(mixed_modB_feats)))

110


137

In [ ]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.3918283275921343

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.030503231964936695

### 1-1 only

In [ ]:
filt_corr_ind_A = []
filt_corr_ind_B = []
seen = set()
for ind_A, ind_B in zip(mixed_modA_feats, mixed_modB_feats):
    if ind_A in kept_modA_feats:
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
    elif ind_A not in seen:  # only keep one if it's over count X
        seen.add(ind_A)
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
num_unq_pairs = len(list(set(filt_corr_ind_A)))
print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
print("num feats after filt: ", len(filt_corr_ind_A))

% unique:  1.0
num feats after filt:  110


In [ ]:
svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")

0.1334318481453728

### take more rand PAIRED subsets

In [ ]:
num_feats = len(list(set(mixed_modA_feats)))
scores = []
for i in range(100):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = svcca(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.12120409428401985

In [ ]:
p_value = np.mean(np.array(scores) >= paired_svcca)
p_value

0.0

In [ ]:
import statistics
variance_value = statistics.variance(scores)
print(f"Variance: {variance_value}")

Variance: 0.008876005652489442


In [ ]:
import math
std_dev = statistics.stdev(scores)
n = len(scores)
standard_error = std_dev / math.sqrt(n)
print(f"Standard Error: {standard_error}")

Standard Error: 0.009421255570511523


In [ ]:
scores[-1]

0.1844498694571838

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6529003926013646

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.06909158685508643

### rsa

In [ ]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

In [ ]:
paired_score = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_score

0.14180832304935834

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          representational_similarity_analysis, shapereq_bool=True)

-0.002496852987186056

In [ ]:
num_feats = len(list(set(mixed_modA_feats)))
scores = []
for i in range(100):
    modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()
    modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
    X_subset = weight_matrix_np[modA_feats, :]
    Y_subset = weight_matrix_2[modB_feats, :]

    score = representational_similarity_analysis(X_subset, Y_subset, "nd")
    scores.append(score)
sum(scores) / len(scores)

0.03523016008103479

In [ ]:
p_value = np.mean(np.array(scores) >= paired_score)
p_value

0.0

In [ ]:
import statistics
variance_value = statistics.variance(scores)
print(f"Variance: {variance_value}")

Variance: 0.000433885603107303


In [ ]:
import math
std_dev = statistics.stdev(scores)
n = len(scores)
standard_error = std_dev / math.sqrt(n)
print(f"Standard Error: {standard_error}")

Standard Error: 0.0020829920861762845


In [ ]:
scores[-1]

0.06836161013229688

In [ ]:
new_vals = []
seen = set()
for ind_A, ind_B in zip(modA_feats, modB_feats):
    if ind_A not in seen:
        seen.add(ind_A)
        val = highest_correlations_values_AB[ind_B]
        new_vals.append(val)
sum(new_vals) / len(new_vals)

0.6700048738657826

In [ ]:
shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.07650087165095348

# match by semantic subspace- appr 2

APPR2: match by separately finding semantics from each then trying

## get mod A labels

In [ ]:
# store feature : lst of top strs
feat_snip_dict = {}
samp_m = 5

for feature_idx in range(feature_acts_A.shape[-1]):
    if feature_idx % 5000 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_A, feature_idx, samp_m, batch_tokens=inputs["input_ids"])
    feat_snip_dict[feature_idx] = store_top_sequences_asLst(ds_top_acts_indices, ds_top_acts_values, inputs["input_ids"])

Feature:  0
Feature:  5000
Feature:  10000
Feature:  15000
Feature:  20000
Feature:  25000
Feature:  30000


In [ ]:
feature_top_samps_lst = [{"feature": feature, "strings": strings} for feature, strings in feat_snip_dict.items()]

In [ ]:
fList_model_A = []
for feat_dict in feature_top_samps_lst:
    out_str = ''
    for text in feat_dict['strings']:
        result = extract_tagged_word(text)
        out_str += result + ', '
    fList_model_A.append(out_str)

In [ ]:
from google.colab import files

In [ ]:
with open('feat_snip_dict_A.pkl', 'wb') as f:
    pickle.dump(feat_snip_dict, f)
files.download('feat_snip_dict_A.pkl')

with open('fList_model_A.pkl', 'wb') as f:
    pickle.dump(fList_model_A, f)
files.download('fList_model_A.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## she

In [ ]:
keyword = "she"
modB_feats = find_indices_with_keyword(fList_model_B, keyword)
modA_feats = find_indices_with_keyword(fList_model_A, keyword)

In [ ]:
len(modA_feats)

23

In [ ]:
len(modB_feats)

22

In [ ]:
"""
`batched_correlation(reshaped_activations_B, reshaped_activations_A)` :
    highest_correlations_indices_AB contains modA's feats as inds, and modB's feats as vals.
    Use the list with smaller number of features (cols) as the second arg
(reshaped_activations_A, reshaped_activations_B): modB is inds, modA is vals of highest_correlations_indices_AB
"""
subset_inds, subset_vals = batched_correlation(reshaped_activations_A[:, modA_feats],
                                               reshaped_activations_B[:, modB_feats])
subset_inds = subset_inds.detach().cpu().numpy()
subset_vals = subset_vals.detach().cpu().numpy()

num_unq_pairs = len(list(set(subset_inds)))
print("% unique: ", num_unq_pairs / len(subset_inds))

sum(subset_vals) / len(subset_vals)

% unique:  0.3181818181818182


0.3680940195918083

In [ ]:
X_subset = weight_matrix_np[modA_feats]
Y_subset = weight_matrix_2[modB_feats]

In [ ]:
Y_subset.shape

(22, 768)

In [ ]:
len(subset_inds)

22

In [ ]:
subset_inds

array([ 6,  3,  9,  4,  3,  9, 21, 16,  4,  9,  3,  9, 21,  4,  4,  3,  3,
        3,  9,  1,  6,  9])

Even without pairing, bc every feature corresponds to the same keyword (at least in one dataset samp of top 5), it should have high sim

In [ ]:
svcca(X_subset[subset_inds], Y_subset, "nd")

0.18697225121890598

In [ ]:
# this is the wrong way to do it; subset_inds contains vals of A and is size of B
svcca(X_subset[0:len(subset_inds)], Y_subset[subset_inds], "nd")

0.3640679447855946

In [ ]:
num_feats = len(modB_feats)
score_rand(weight_matrix_np, weight_matrix_2, num_feats,
                                          svcca, shapereq_bool=True)

0.16337664939514743

In [ ]:
all_rand_scores = []
for i in range(1000):
    rand_modA_feats = np.random.randint(low=0, high=weight_matrix_np.shape[0], size=num_feats).tolist()
    rand_modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()

    score = svcca(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats], "nd")
    all_rand_scores.append(score)
sum(all_rand_scores) / len(all_rand_scores)

0.17712704318394157

## he

In [ ]:
keyword = "he"
modB_feats = find_indices_with_keyword(fList_model_B, keyword)
modA_feats = find_indices_with_keyword(fList_model_A, keyword)

In [ ]:
len(modA_feats)

93

In [ ]:
len(modB_feats)

82

In [ ]:
"""
`batched_correlation(reshaped_activations_B, reshaped_activations_A)` :
    highest_correlations_indices_AB contains modA's feats as inds, and modB's feats as vals.
    Use the list with smaller number of features (cols) as the second arg
(reshaped_activations_A, reshaped_activations_B): modB is inds, modA is vals of highest_correlations_indices_AB
"""
subset_inds, subset_vals = batched_correlation(reshaped_activations_A[:, modA_feats],
                                               reshaped_activations_B[:, modB_feats])
subset_inds = subset_inds.detach().cpu().numpy()
subset_vals = subset_vals.detach().cpu().numpy()

num_unq_pairs = len(list(set(subset_inds)))
print("% unique: ", num_unq_pairs / len(subset_inds))

sum(subset_vals) / len(subset_vals)

% unique:  0.2926829268292683


0.4051277958674402

In [ ]:
X_subset = weight_matrix_np[modA_feats]
Y_subset = weight_matrix_2[modB_feats]

Even without pairing, bc every feature corresponds to the same keyword (at least in one dataset samp of top 5), it should have high sim

In [ ]:
len(subset_inds)

82

In [ ]:
X_subset.shape

(93, 512)

In [ ]:
paired_svcca = svcca(X_subset[subset_inds], Y_subset, "nd")
paired_svcca

0.2014914636058596

In [ ]:
num_feats = len(modB_feats)

all_rand_scores = []
for i in range(1000):
    rand_modA_feats = np.random.randint(low=0, high=weight_matrix_np.shape[0], size=num_feats).tolist()
    rand_modB_feats = np.random.randint(low=0, high=weight_matrix_2.shape[0], size=num_feats).tolist()

    score = svcca(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats], "nd")
    all_rand_scores.append(score)
sum(all_rand_scores) / len(all_rand_scores)

0.08892504477860255

In [ ]:
p_value = np.mean(np.array(all_rand_scores) >= paired_svcca)
p_value

0.069

## combos of numerics

In [45]:
mixed_modA_feats = []
mixed_modB_feats = []
keywords = [
    "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
    "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen", "twenty",
    "hundred", "thousand", "million", "billion", "trillion",
    "integer", "fraction", "decimal", "percentage", "ratio",
    "numeral", "digit", "prime", "even", "odd",
    "sum", "difference", "product", "factor", "multiple",
    "total", "count", "measure", "dozen", "score", "unit"
]

for kw in keywords:
    modB_feats = find_indices_with_keyword(fList_model_B, kw)
    modA_feats = find_indices_with_keyword(fList_model_A, kw)
    mixed_modA_feats.extend(modA_feats)
    mixed_modB_feats.extend(modB_feats)

mixed_modA_feats = list(set(mixed_modA_feats))
mixed_modB_feats = list(set(mixed_modB_feats))
print( len(list(set(mixed_modA_feats))) )
len(list(set(mixed_modB_feats)))

648


417

In [63]:
subset_inds, subset_vals = batched_correlation(reshaped_activations_A[:, mixed_modA_feats],
                                               reshaped_activations_B[:, mixed_modB_feats])
subset_inds = subset_inds.detach().cpu().numpy()
subset_vals = subset_vals.detach().cpu().numpy()

num_unq_pairs = len(list(set(subset_inds)))
print("% unique: ", num_unq_pairs / len(subset_inds))
sum(subset_vals) / len(subset_vals)

% unique:  0.328537170263789


0.45470770497804136

In [72]:
X_subset = weight_matrix_np[mixed_modA_feats]
Y_subset = weight_matrix_2[mixed_modB_feats]

paired_svcca = svcca(X_subset[subset_inds], Y_subset, "nd")
paired_svcca

0.1478672734824761

### 1-1 only

In [65]:
subset_sorted_feat_counts = Counter(subset_inds).most_common()
subset_kept_modA_feats = [feat_ID for feat_ID, count in subset_sorted_feat_counts if count <= 1]

filt_corr_ind_A = []
filt_corr_ind_B = []
seen = set()
for ind_B, ind_A in enumerate(subset_inds):
    if ind_A in subset_kept_modA_feats:
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
    elif ind_A not in seen:  # only keep one if it's over count X
        seen.add(ind_A)
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
num_unq_pairs = len(list(set(filt_corr_ind_A)))
print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))

% unique:  1.0


In [79]:
paired_score = svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")
paired_score

0.1824494416515855

In [78]:
X_subset = weight_matrix_np[filt_corr_ind_A]
Y_subset = weight_matrix_2[filt_corr_ind_B]

all_rand_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)
sum(all_rand_scores) / len(all_rand_scores)

0.059609094905483585

In [80]:
np.mean(np.array(all_rand_scores) >= paired_score)

0.01

In [76]:
num_feats = len(list(set(filt_corr_ind_A)))
score_rand(10, weight_matrix_np, weight_matrix_2, num_feats, svcca, True)

0.06936413905459124

### manyBto1A

In [56]:
subset_inds, subset_vals = batched_correlation(reshaped_activations_B[:, mixed_modB_feats],
                                               reshaped_activations_A[:, mixed_modA_feats])
subset_inds = subset_inds.detach().cpu().numpy()
subset_vals = subset_vals.detach().cpu().numpy()

num_unq_pairs = len(list(set(subset_inds)))
print("% unique: ", num_unq_pairs / len(subset_inds))
sum(subset_vals) / len(subset_vals)

% unique:  0.22530864197530864


0.4241289074423458

In [57]:
X_subset = weight_matrix_np[mixed_modA_feats]
Y_subset = weight_matrix_2[mixed_modB_feats]
paired_svcca = svcca(X_subset, Y_subset[subset_inds], "nd")
paired_svcca

0.01080813121097554

#### 1-1 only

In [60]:
subset_sorted_feat_counts = Counter(subset_inds).most_common()
subset_kept_modB_feats = [feat_ID for feat_ID, count in subset_sorted_feat_counts if count <= 1]

filt_corr_ind_A = []
filt_corr_ind_B = []
seen = set()
for ind_A, ind_B in enumerate(subset_inds):
    if ind_B in subset_kept_modB_feats:
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
    elif ind_B not in seen:  # only keep one if it's over count X
        seen.add(ind_B)
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
num_unq_pairs = len(list(set(filt_corr_ind_A)))
print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))

% unique:  1.0


In [61]:
svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")

0.05286786914369134

In [62]:
X_subset = weight_matrix_np[filt_corr_ind_A]
Y_subset = weight_matrix_2[filt_corr_ind_B]

shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)

0.07489191612976505

## combos of people

In [89]:
mixed_modA_feats = []
mixed_modB_feats = []
keywords = [
    "man", "girl", "boy", "kid", "dad", "mom", "son", "sis", "bro",
    "pal", "mate", "boss", "chief", "cop", "guide", "priest", "king",
    "queen", "duke", "lord", "friend", "judge", "clerk", "coach", "team",
    "crew", "staff", "nurse", "doc", "vet", "cook", "maid", "clown",
    "star", "clan", "host", "guest", "peer", "guard", "boss", "spy",
    "fool", "punk", "nerd", "jock", "chief", "folk", "crowd"
]

for kw in keywords:
    modB_feats = find_indices_with_keyword(fList_model_B, kw)
    modA_feats = find_indices_with_keyword(fList_model_A, kw)
    mixed_modA_feats.extend(modA_feats)
    mixed_modB_feats.extend(modB_feats)

mixed_modA_feats = list(set(mixed_modA_feats))
mixed_modB_feats = list(set(mixed_modB_feats))
print( len(list(set(mixed_modA_feats))) )
len(list(set(mixed_modB_feats)))

197


400

In [90]:
subset_inds, subset_vals = batched_correlation(reshaped_activations_A[:, mixed_modA_feats],
                                               reshaped_activations_B[:, mixed_modB_feats])
subset_inds = subset_inds.detach().cpu().numpy()
subset_vals = subset_vals.detach().cpu().numpy()

num_unq_pairs = len(list(set(subset_inds)))
print("% unique: ", num_unq_pairs / len(subset_inds))
sum(subset_vals) / len(subset_vals)

% unique:  0.2075


0.37552560058888046

In [91]:
X_subset = weight_matrix_np[mixed_modA_feats]
Y_subset = weight_matrix_2[mixed_modB_feats]

paired_svcca = svcca(X_subset[subset_inds], Y_subset, "nd")
paired_svcca

0.13538255822749162

### 1-1 only

In [97]:
subset_sorted_feat_counts = Counter(subset_inds).most_common()
subset_kept_modA_feats = [feat_ID for feat_ID, count in subset_sorted_feat_counts if count == 1]

filt_corr_ind_A = []
filt_corr_ind_B = []
seen = set()
for ind_B, ind_A in enumerate(subset_inds):
    if ind_A in subset_kept_modA_feats:
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
    elif ind_A not in seen:  # only keep one if it's over count X
        seen.add(ind_A)
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
num_unq_pairs = len(list(set(filt_corr_ind_A)))
print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))

% unique:  1.0


In [98]:
paired_score = svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")
paired_score

0.19667233372087772

In [94]:
X_subset = weight_matrix_np[filt_corr_ind_A]
Y_subset = weight_matrix_2[filt_corr_ind_B]

all_rand_scores = shuffle_rand(1000, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)
sum(all_rand_scores) / len(all_rand_scores)

0.08883765117462544

In [95]:
np.mean(np.array(all_rand_scores) >= paired_score)

0.067

In [96]:
num_feats = len(list(set(filt_corr_ind_A)))
score_rand(10, weight_matrix_np, weight_matrix_2, num_feats, svcca, True)

0.08164697724405635

## combos of nature

In [99]:
mixed_modA_feats = []
mixed_modB_feats = []
keywords = [
    "tree", "grass", "bush", "plant", "stone", "rock", "cliff", "hill",
    "dirt", "sand", "mud", "wind", "storm", "rain", "cloud", "sun",
    "moon", "star", "leaf", "branch", "twig", "root", "bark", "seed",
    "wave", "tide", "lake", "pond", "creek", "sea", "wood", "field",
    "shore", "snow", "ice", "flame", "fire", "fog", "dew", "hail",
    "sky", "earth", "glade", "cave", "peak", "ridge", "dust", "air",
    "mist", "heat"
]

for kw in keywords:
    modB_feats = find_indices_with_keyword(fList_model_B, kw)
    modA_feats = find_indices_with_keyword(fList_model_A, kw)
    mixed_modA_feats.extend(modA_feats)
    mixed_modB_feats.extend(modB_feats)

mixed_modA_feats = list(set(mixed_modA_feats))
mixed_modB_feats = list(set(mixed_modB_feats))
print( len(list(set(mixed_modA_feats))) )
len(list(set(mixed_modB_feats)))

169


304

In [100]:
subset_inds, subset_vals = batched_correlation(reshaped_activations_A[:, mixed_modA_feats],
                                               reshaped_activations_B[:, mixed_modB_feats])
subset_inds = subset_inds.detach().cpu().numpy()
subset_vals = subset_vals.detach().cpu().numpy()

num_unq_pairs = len(list(set(subset_inds)))
print("% unique: ", num_unq_pairs / len(subset_inds))
sum(subset_vals) / len(subset_vals)

% unique:  0.22697368421052633


0.3662683577123588

In [101]:
X_subset = weight_matrix_np[mixed_modA_feats]
Y_subset = weight_matrix_2[mixed_modB_feats]

paired_svcca = svcca(X_subset[subset_inds], Y_subset, "nd")
paired_svcca

0.22030600501351066

### 1-1 only

In [102]:
subset_sorted_feat_counts = Counter(subset_inds).most_common()
subset_kept_modA_feats = [feat_ID for feat_ID, count in subset_sorted_feat_counts if count == 1]

filt_corr_ind_A = []
filt_corr_ind_B = []
seen = set()
for ind_B, ind_A in enumerate(subset_inds):
    if ind_A in subset_kept_modA_feats:
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
    elif ind_A not in seen:  # only keep one if it's over count X
        seen.add(ind_A)
        filt_corr_ind_A.append(ind_A)
        filt_corr_ind_B.append(ind_B)
num_unq_pairs = len(list(set(filt_corr_ind_A)))
print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))

% unique:  1.0


In [103]:
paired_score = svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")
paired_score

0.21105044195342468

In [104]:
X_subset = weight_matrix_np[filt_corr_ind_A]
Y_subset = weight_matrix_2[filt_corr_ind_B]

all_rand_scores = shuffle_rand(1000, X_subset, Y_subset, Y_subset.shape[0],
                                          svcca, shapereq_bool=True)
sum(all_rand_scores) / len(all_rand_scores)

0.09767672743920548

In [105]:
np.mean(np.array(all_rand_scores) >= paired_score)

0.073

In [106]:
num_feats = len(list(set(filt_corr_ind_A)))
score_rand(10, weight_matrix_np, weight_matrix_2, num_feats, svcca, True)

0.12001724986696775

# compare feats A "she" by corrB vs search

In [ ]:
keyword = "she"
modB_feats = find_indices_with_keyword(fList_model_B, keyword)
modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
# X_subset = weight_matrix_np[modA_feats, :]
# Y_subset = weight_matrix_2[modB_feats, :]

In [ ]:
modA_feats_from_search = find_indices_with_keyword(fList_model_A, keyword)

In [ ]:
len(modA_feats_from_search)

23

In [ ]:
set(modA_feats).intersection(set(modA_feats_from_search))

{2577, 4262, 10853}

In [ ]:
modA_feats_from_search

[47,
 1272,
 1915,
 2577,
 4262,
 5004,
 6175,
 6356,
 8408,
 10853,
 15281,
 15563,
 16245,
 17800,
 20038,
 22099,
 23107,
 29124,
 29745,
 29943,
 30310,
 30608,
 31417]

In [ ]:
modA_feats

[23687,
 23687,
 6219,
 10178,
 23687,
 8242,
 14815,
 23687,
 4262,
 7882,
 23687,
 2310,
 24468,
 2232,
 2604,
 2577,
 23687,
 13908,
 10853,
 23687,
 23687,
 10321]

In [ ]:
for fID in range(10):
    feature_idx_A = modA_feats_from_search[fID]
    print('Model A Feature: ', feature_idx_A)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_A, feature_idx_A, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

Model A Feature:  47


batchID: 29, Act = 0.74, Seq = " look at his wife, she was standing there."
batchID: 79, Act = 0.73, Seq = " in our secular culture, they are no longer a"
batchID: 52, Act = 0.61, Seq = " relationship with the EU, they write: “The"
batchID: 85, Act = 0.59, Seq = "ed with DUI, she was taken to the"
batchID: 35, Act = 0.59, Seq = " in Europe and elsewhere, they said.\n\n"

Model A Feature:  1272


batchID: 41, Act = 0.67, Seq = "". I asked him if he wanted me to sign"
batchID: 39, Act = 0.56, Seq = " to help ourselves sometimes," he said.\n\n"
batchID: 38, Act = 0.50, Seq = "\n\nSheen says he’s “dis"
batchID: 66, Act = 0.40, Seq = "old said sheepishly as she made a beeline"
batchID: 41, Act = 0.29, Seq = "\n"I asked if he was able was able"

Model A Feature:  1915


...

batchID: 84, Act = 0.68, Seq = " with special needs is that they often have to spend"
batchID: 85, Act = 0.67, Seq = " when a cop noticed that she, too, seemed"
batchID: 93, Act = 0.61, Seq = " pose, the notion that they are far more common"
batchID: 13, Act = 0.57, Seq = " example, just announced that they’re working with"
batchID: 29, Act = 0.56, Seq = " Trump’s speculation that she didn’t speak"

Model A Feature:  8408


batchID: 84, Act = 0.60, Seq = "Last month, the Seattle"
batchID: 25, Act = 0.54, Seq = " being called Osama Bin Laden to Paki-"
batchID: 48, Act = 0.51, Seq = "Israel Accused of Suppressing"
batchID: 38, Act = 0.46, Seq = "Martin Sheen says he’"
batchID: 38, Act = 0.45, Seq = "Martin Sheen says he"

Model A Feature:  10853


batchID: 38, Act = 0.04, Seq = "Martin Sheen says he’"
batchID: 24, Act = 0.03, Seq = " be better than your drive home," said Black."
batchID: 60, Act = 0.00, Seq = ", the former hedge fund manager under fire for buying"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince,"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince"

In [ ]:
for fID in range(10):
    feature_idx_A = modA_feats[fID]
    print('Model A Feature: ', feature_idx_A)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_A, feature_idx_A, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

Model A Feature:  23687


batchID: 0, Act = 5.28, Seq = " the team.\n\nHe said it was a"
batchID: 63, Act = 5.27, Seq = " aged 60.\n\nHe had a progressive lung"
batchID: 96, Act = 5.27, Seq = ",” Quinn said. “He was on the plan"
batchID: 98, Act = 5.26, Seq = " stand-up days. He alleges that this was"
batchID: 6, Act = 5.25, Seq = " from credit card machines. He says he plans to"

Model A Feature:  23687


batchID: 0, Act = 5.28, Seq = " the team.\n\nHe said it was a"
batchID: 63, Act = 5.27, Seq = " aged 60.\n\nHe had a progressive lung"
batchID: 96, Act = 5.27, Seq = ",” Quinn said. “He was on the plan"
batchID: 98, Act = 5.26, Seq = " stand-up days. He alleges that this was"
batchID: 6, Act = 5.25, Seq = " from credit card machines. He says he plans to"

Model A Feature:  6219


...

batchID: 0, Act = 5.28, Seq = " the team.\n\nHe said it was a"
batchID: 63, Act = 5.27, Seq = " aged 60.\n\nHe had a progressive lung"
batchID: 96, Act = 5.27, Seq = ",” Quinn said. “He was on the plan"
batchID: 98, Act = 5.26, Seq = " stand-up days. He alleges that this was"
batchID: 6, Act = 5.25, Seq = " from credit card machines. He says he plans to"

Model A Feature:  4262


batchID: 59, Act = 2.04, Seq = " SneakyStyL Heirophant pr"
batchID: 59, Act = 1.86, Seq = "13 RohesiaCrow Heisman1 Rosenc"
batchID: 16, Act = 1.67, Seq = " REUTERS/Maxim Shemetov\n\n"
batchID: 93, Act = 1.59, Seq = " Is Entitled to Be Heard?” Daniel Bren"
batchID: 72, Act = 1.51, Seq = "He was a few blocks"

Model A Feature:  7882


batchID: 68, Act = 4.96, Seq = "Oh my goodness,” the victim's mother, Shannon"
batchID: 39, Act = 4.80, Seq = " driving.\n\nThe victim is expected to be"
batchID: 16, Act = 4.68, Seq = " it too is a frequent victim of cyber crime in"
batchID: 16, Act = 4.57, Seq = " made the country a major victim of the global ext"
batchID: 88, Act = 4.42, Seq = ". The mayor had been victimized by drug smug"

# compare feats A "he" by corrB vs search

In [ ]:
keyword = "he"
modB_feats = find_indices_with_keyword(fList_model_B, keyword)
modA_feats = get_values_from_indices(modB_feats, highest_correlations_indices_AB)
# X_subset = weight_matrix_np[modA_feats, :]
# Y_subset = weight_matrix_2[modB_feats, :]

In [ ]:
modA_feats_from_search = find_indices_with_keyword(fList_model_A, keyword)

In [ ]:
len(modA_feats_from_search)

93

In [ ]:
set(modA_feats).intersection(set(modA_feats_from_search))

{2232, 3743, 4262, 10178, 11573, 18693, 19853, 19938, 23687, 24280, 31577}

In [ ]:
for fID in range(10):
    feature_idx_A = modA_feats_from_search[fID]
    print('Model A Feature: ', feature_idx_A)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_A, feature_idx_A, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

Model A Feature:  149


batchID: 16, Act = 0.40, Seq = " working and stable,” he said.\n\nOther"
batchID: 16, Act = 0.38, Seq = " are working and stable,” he said.\n\n"
batchID: 95, Act = 0.33, Seq = " uphill road,” he said then.\n\n"
batchID: 55, Act = 0.28, Seq = " way government works," he said. "You're"
batchID: 6, Act = 0.26, Seq = "\n\nInstead, he says he’s paying"

Model A Feature:  752


batchID: 5, Act = 0.02, Seq = " censorship, “blasphemy laws” and"
batchID: 84, Act = 0.00, Seq = " play area designed for special needs kids, claiming its"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince, Haiti"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince,"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince"

Model A Feature:  1272


...

batchID: 98, Act = 3.58, Seq = " says these types of outbursts were commonplace from"
batchID: 0, Act = 0.48, Seq = "Port-au-Prince, Haiti (CNN"
batchID: 47, Act = 0.28, Seq = " newly opened car air freshener, not everything"
batchID: 65, Act = 0.28, Seq = " Tapert and star Bruce Campbell are rolling cameras"
batchID: 5, Act = 0.26, Seq = " censorship, “blasphemy laws” and"

Model A Feature:  3743


batchID: 96, Act = 1.16, Seq = " March 6.\n\nHe was cleared before training"
batchID: 63, Act = 1.12, Seq = " aged 60.\n\nHe had a progressive lung"
batchID: 53, Act = 1.10, Seq = " the incident.\n\nHeavy police force has"
batchID: 72, Act = 1.06, Seq = " writer and activist\n\nHe felt like a suspect"
batchID: 5, Act = 0.99, Seq = "Clements disagreed. “He knew the material would"

Model A Feature:  4227


batchID: 50, Act = 0.54, Seq = " 27 dead, Congress has not enacted substantial legislative changes"
batchID: 1, Act = 0.53, Seq = ", said the campaign has not made a final decision"
batchID: 25, Act = 0.36, Seq = ", it seems life has not been the same for"
batchID: 21, Act = 0.33, Seq = " said. His name has not been released.\n"
batchID: 6, Act = 0.24, Seq = " the Pacific Northwest. And he’s calling on"

In [ ]:
for fID in range(10):
    feature_idx_A = modA_feats[fID]
    print('Model A Feature: ', feature_idx_A)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_A, feature_idx_A, samp_m, batch_tokens=inputs["input_ids"])
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=inputs["input_ids"])

Model A Feature:  25476


batchID: 36, Act = 0.46, Seq = "Whenever something momentous happens"
batchID: 3, Act = 0.45, Seq = " those musical entities that transcends language. It’"
batchID: 36, Act = 0.43, Seq = "Whenever something momentous happens in"
batchID: 50, Act = 0.42, Seq = " cycle, gun rights organizations spent 34 times more on"
batchID: 72, Act = 0.41, Seq = " a Hamilton Police officer who seemed not to realize who"

Model A Feature:  23687


batchID: 0, Act = 5.28, Seq = " the team.\n\nHe said it was a"
batchID: 63, Act = 5.27, Seq = " aged 60.\n\nHe had a progressive lung"
batchID: 96, Act = 5.27, Seq = ",” Quinn said. “He was on the plan"
batchID: 98, Act = 5.26, Seq = " stand-up days. He alleges that this was"
batchID: 6, Act = 5.25, Seq = " from credit card machines. He says he plans to"

Model A Feature:  23687


...

batchID: 27, Act = 0.88, Seq = " the New York Stock Exchange.\n\nThe Dow"
batchID: 69, Act = 0.87, Seq = " an even more uncertain future\n\nEmotions ran"
batchID: 37, Act = 0.86, Seq = "Executive summary\n\nThis paper reviews"
batchID: 13, Act = 0.85, Seq = " sometimes things are more complicated.\n\nAdBlock"
batchID: 56, Act = 0.85, Seq = "’t know for sure. But the trackers"

Model A Feature:  27004


batchID: 44, Act = 4.49, Seq = "It straightforwardly reported his defection from the Democrats"
batchID: 90, Act = 4.11, Seq = " too much it shouldn't dehydrate you."
batchID: 0, Act = 0.00, Seq = "Port-au-Prince, Haiti"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince,"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince"

Model A Feature:  10984


batchID: 57, Act = 0.19, Seq = "Media playback is unsupported on your"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince, Haiti"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince"
batchID: 0, Act = 0.00, Seq = "Port-au-Prince, Haiti ("
batchID: 0, Act = 0.00, Seq = "Port-au-Prince,"

# sim metric on WxA

In [ ]:
# takes too long
# svcca(reshaped_activations_A.t(), reshaped_activations_B.t(), "nd")

In [ ]:
# takes too long
# svcca(reshaped_activations_A[:30000].t(), reshaped_activations_B[:30000].t(), "nd")

In [ ]:
reshaped_activations_A[:1000].t().shape

torch.Size([32768, 1000])

In [ ]:
svcca(reshaped_activations_A[:1000].t(), reshaped_activations_B[:1000].t(), "nd")

0.003415757178864885

In [ ]:
svcca(reshaped_activations_A[:1000].t()[highest_correlations_indices_AB], reshaped_activations_B[:1000].t(), "nd")

0.5541710603648216

In [ ]:
svcca(reshaped_activations_A[:5000].t()[highest_correlations_indices_AB], reshaped_activations_B[:5000].t(), "nd")

0.47164446194747334